<a href="https://colab.research.google.com/github/ella417/NLP/blob/main/NLP_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Attention
- In this assignment, you have to implement the attention mechanism for the machine translation task.

- Problems
  - 1. Implement Dot Product Attention (10 pts)
  - 2. Implement Attention in Batch (12 pts)
  - 3. Implement Seq2seq with Attention (14 pts)
  - 4. Implement Transformer-like Attention (14 pts)
    - Self-attention
    - Cross-attention

- Submission
  - You have to copy and paste your answer to ``NLP_Assignment_4.py`` file and submit it to Cyber Campus
    - Submssion file name has to be ``NLP_Assignment_4_{your_student_id}.py``
    - { } character is just placeholder, so don't inlcude it in your filename.

- CAUTION:
  - You have to implement most of the functions with **vectorized matrix multiplication**, not with for loop



- If you find any error, please do not hesitate to report or make a question on Cyber Campus
    - Don't waste too much time on solving the error. The code is not thoroughly checked, and the error can be not your fault.

In [1]:
# If you are in Colab, install transformers
!pip -q install transformers
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.0 MB/s eta 0:00:00


In [2]:
# Download the py file for submission
!wget https://raw.githubusercontent.com/jdasam/aat3020/2025/NLP_Assignment_4.py
!wget https://raw.githubusercontent.com/jdasam/aat3020/2025/assignment4_pre_defined.py

--2025-06-27 11:42:17--  https://raw.githubusercontent.com/jdasam/aat3020/2025/NLP_Assignment_4.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41934 (41K) [text/plain]
Saving to: ‘NLP_Assignment_4.py’

NLP_Assignment_4.py 100%[===================>]  40.95K  --.-KB/s    in 0.001s  

2025-06-27 11:42:17 (53.8 MB/s) - ‘NLP_Assignment_4.py’ saved [41934/41934]

--2025-06-27 11:42:17--  https://raw.githubusercontent.com/jdasam/aat3020/2025/assignment4_pre_defined.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13767 (13K) [text/

In [3]:
import torch
import torch.nn as nn
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import koreanize_matplotlib
import numpy as np

from torch.nn.utils.rnn import PackedSequence, pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast

import os

# Below helps to run tokenizer with multiprocessing
os.environ["TOKENIZERS_PARALLELISM"] = "true"

#### Vectorization

In [4]:
'''
This is the example of vectorization of dot product for two different sequence length.
'''

e_states = torch.randn(100, 16)
d_states = torch.randn(80, 16)

dot_product = torch.mm(e_states, d_states.permute(1,0)) # (100, 16) x (16, 80) = (100, 80)
dot_product

tensor([[ 4.6967e+00,  4.4821e-01, -3.3510e+00,  ...,  5.7956e+00,
          2.1540e+00,  1.4854e+00],
        [-9.7154e-01, -1.3340e+01,  6.9144e+00,  ...,  1.2419e+00,
         -2.6216e+00, -1.9333e+00],
        [ 8.6621e-01, -9.9964e-02, -1.6763e+00,  ...,  2.3058e-01,
         -4.1958e+00,  5.3794e+00],
        ...,
        [-4.9264e+00, -8.6890e-01,  5.3011e+00,  ..., -7.1469e+00,
          8.2680e+00,  6.9801e+00],
        [ 5.1104e+00,  1.4988e+01, -7.1690e+00,  ..., -6.0676e-01,
         -3.5771e+00, -4.6799e+00],
        [ 3.8025e+00,  2.4862e+00, -8.1009e-01,  ..., -9.4187e-03,
         -1.7990e+00, -1.1718e+00]])

## Problem 1: Implement Dot Product Attention

- Optimizing computation time is really important
    - Use `torch.mm()` or `torch.matmul()`
    - `torch.mm(a, b)` is a function for calculating matrix multiplcation of two matrices `a` and `b`
        - `a` and `b` has to be 2-dim tensors
        - `a.shape[1]` has to be equal to `b.shape[0]`
    - `torch.matmul()` is a function for matrix multiplication but with broadcasting
        - https://pytorch.org/docs/stable/generated/torch.matmul.html
        - It has less restriction on its input shape.
            - It automatically matches the dimension of two tensors following some rules
            - Therefore, it is a bit risky to use this funciton if you don't understand how it works
- **DO NOT** use element-wise product or for loop!

### Hint: Dot product as matrix multiplcation.

- Let's say there are two vector, $u=\begin{bmatrix}-3 \\ 2 \\ 1\end{bmatrix}$ and $v = \begin{bmatrix} 5 \\ 4 \\ 6\end{bmatrix}$
    - The dot product of the two vectors is $(-3 \times 5) + (2 \times 4) + (1 \times 6) = 1$
    - It is equivalent to $u^T \times v$
        - In this case $u\in\mathbb{R}^{3\times1}$ and $v\in\mathbb{R}^{3\times1}$
- In PyTorch, this can be described as below:
    - `u = torch.Tensor([-3, 2, 1])`
    - `v = torch.Tensor([5, 4, 6])`
    - Dot product of u and v can be calculated by one of belows:
        - `torch.mm(u.unsqueeze(0), v.unsqueeze(1))`
            - `u.unsqueeze(0).shape == [1, 3]`
            - `v.unsqueeze(1).shape == [3, 1]`
            - `unsqueeze()` returns a new tensor with a dimension of size one inserted at the specified position.
            - The result has shape of [1,1]
        - `torch.matmul(u, v)`
        - `u @ v`
            - `@` denotes matrix multiplication, which was introduced from Python 3.5
        - `(u * v).sum()`
            - This will be much slower than others, because it first do element-wise multiplcation

In [5]:
'''
Hint: Dot product as matrix multiplcation.
'''

u = torch.Tensor([-3, 2, 1])
v = torch.Tensor([5, 4, 6])

print(f"Result of (u * v).sum() is {(u * v).sum()}. This computation is much slower than others because it use element-wise multiplication instead of matrix multiplication")
print(f"Result of torch.mm(u.unsqueeze(0), v.unsqueeze(1)) is {torch.mm(u.unsqueeze(0), v.unsqueeze(1))}")
print(f"Result of torch.matmul(u, v) is {torch.matmul(u, v)}")
print(f"Result of u @ v is {u @ v}")

Result of (u * v).sum() is -1.0. This computation is much slower than others because it use element-wise multiplication instead of matrix multiplication
Result of torch.mm(u.unsqueeze(0), v.unsqueeze(1)) is tensor([[-1.]])
Result of torch.matmul(u, v) is -1.0
Result of u @ v is -1.0


### 1-1 Get attention score with dot product (4 pts)
- From a sequence of key and a single query

In [6]:
def get_attention_score_for_a_single_query(keys, query):
  '''
  This function returns an attention score for each vector in keys for a given query.
  You can regard 'keys' as hidden states over timestep of Encoder, while query is a hidden state of specific time step of Decoder
  Name 'keys' are used because it is used for calculating attention score (match rate between given vector and query).

  For every C-dimensional vector key, the attention score is a dot product between the key and the query vector.

  Arguments:
    keys (torch.Tensor): Has a shape of [T, C]. These are vectors that a query wants attend to
    query (torch.Tensor): Has a shape of [C]. This is a vector that attends to other set of vectors (keys and values)

  Output:
    attention_score (torch.Tensor): The attention score in real number that represent how much does query have to attend to each vector in keys
                                    Has a shape of [T]

    attention_score[i] has to be a dot product value between keys[i] and query


  TODO: Complete this sentence using torch.mm (matrix multiplication)
  Hint: You can use atensor.unsqueeze(dim) to expand a dimension (with a diemsion of length 1) without changing item value of the tensor.
  '''

  expanded_query = query.unsqueeze(1) # [C] -> [C, 1]
  attention_score_matrix = torch.mm(keys, expanded_query) # [T, C] x [C, 1] = [T, 1]
  attention_score = attention_score_matrix.squeeze(1) # [T, 1] -> [T]

  return attention_score


torch.manual_seed(0)
num_t = 23
h_size = 16

keys = torch.randn(num_t, h_size)
query = torch.randn(h_size)

att_score = get_attention_score_for_a_single_query(keys, query)
att_score

tensor([-3.0786,  2.1729,  1.7950, -5.0503,  3.3254,  0.2828, -0.9800, -1.8868,
         0.2550,  2.9389, -0.1799, -1.0586,  0.1465, -0.9441,  0.8888, -3.8108,
        -2.5662, -1.1660, -2.2327,  2.7087, -0.5800,  8.7984,  4.3816])

In [7]:
'''
Test Case
'''
assert att_score.ndim == 1 and len(att_score) == num_t, "Error: Check output shape"
answer = torch.Tensor([-3.0786,  2.1729,  1.7950, -5.0503,  3.3254,  0.2828, -0.9800, -1.8868,
         0.2550,  2.9389, -0.1799, -1.0586,  0.1465, -0.9441,  0.8888, -3.8108,
        -2.5662, -1.1660, -2.2327,  2.7087, -0.5800,  8.7984,  4.3816])
assert torch.allclose(att_score, answer, atol=1e-4) , "Error: The output value is different"
print("Passed all the cases!")

Passed all the cases!


### 1-2 Get attention weight from score (2pts)
- Convert attention score to attention weight
- Use softmax function

In [8]:
def get_attention_weight_from_score(attention_score):
  '''
  This function converts attention score to attention weight.

  Argument:
    attention_score (torch.Tensor): Tensor of real number. Has a shape of [T]

  Output:
    attention_weight (torch.Tensor): Tensor of real number between 0 and 1. Sum of attention_weight is 1. Has a shape of [T]

  TODO: Complete this function
  '''

  assert attention_score.ndim == 1
  attention_weight = torch.nn.functional.softmax(attention_score, dim=0)

  return attention_weight

att_weight = get_attention_weight_from_score(att_score)
att_weight

tensor([6.7782e-06, 1.2936e-03, 8.8653e-04, 9.4370e-07, 4.0957e-03, 1.9541e-04,
        5.5277e-05, 2.2321e-05, 1.9005e-04, 2.7829e-03, 1.2303e-04, 5.1099e-05,
        1.7052e-04, 5.7296e-05, 3.5821e-04, 3.2593e-06, 1.1314e-05, 4.5893e-05,
        1.5795e-05, 2.2107e-03, 8.2463e-05, 9.7556e-01, 1.1777e-02])

In [9]:
answer = torch.Tensor([0.0000,     0.0013,     0.0009,     0.0000,     0.0041,     0.0002,
            0.0001,     0.0000,     0.0002,     0.0028,     0.0001,     0.0001,
            0.0002,     0.0001,     0.0004,     0.0000,     0.0000,     0.0000,
            0.0000,     0.0022,     0.0001,     0.9756,     0.0118])
assert att_weight.shape == att_score.shape, 'Shape has to be remained the same'
assert att_weight.sum() == 1, "Sum of attention weight has to be 1"
assert torch.allclose(att_weight, answer, atol=1e-4) , "Error: The output value is different"

print("Passed all the cases!")


Passed all the cases!


### 1-3 Get weighted sum (4 pts)
- Using attention weight and values, get the weighted sum
- $a = \sum_{i=1}^{T} \alpha_i v_i$

In [10]:
def get_weighted_sum(values, attention_weight):
  '''
  This function converts attention score to attention weight

  Argument:
    values (torch.Tensor): Has a shape of [T, C]. These are vectors that are used to form attention vector
    attention_weight: Has a shape of [T], which represents the weight for each vector to compose the attention vector

  Output:
    attention_vector (torch.Tensor): Weighted sum of values using the attention weight. Has a shape of [C]

  TODO: Complete this function using torch.mm
  '''

  expanded_attention_weight = attention_weight.unsqueeze(0) # [T] -> [1, T]
  attention_vector_matrix = torch.mm(expanded_attention_weight, values) # [1, T] x [T, C] = [1, C]
  attention_vector = attention_vector_matrix.squeeze(0) # [1, C] -> [C]

  return attention_vector

att_vec = get_weighted_sum(keys, att_weight) # In simple dot-product-attention, key and value are the same
att_vec

tensor([ 0.6280,  3.8540, -0.1042,  0.3148,  0.3711, -0.5095, -0.9663,  1.3295,
         1.9003, -1.2611, -2.2939, -2.0338,  0.8757, -0.6726,  1.9071, -1.0711])

In [11]:
answer = torch.Tensor([ 0.6280,  3.8540, -0.1042,  0.3148,  0.3711, -0.5095, -0.9663,  1.3295,
         1.9003, -1.2611, -2.2939, -2.0338,  0.8757, -0.6726,  1.9071, -1.0711])
assert att_vec.shape == query.shape, 'Shape has to be remained the same'
assert torch.allclose(att_vec, answer, atol=1e-4) , "Error: The output value is different"
print("Passed all the cases")


Passed all the cases


## Problem 2: Attention in Batch ( 16 pts)
- In this problem, you have to calculate attention with batch
- You can use `torch.bmm()` for batch matrix multiplication https://pytorch.org/docs/stable/generated/torch.bmm.html
    - `torch.bmm()` takes two 3-dim tensor as its input
    - Each tensor has to be 3-dim (atensor.ndim==3)


In [12]:
'''
Hint for Problem 2

You can calculate matrix multiplication of matrices in batch effectively using torch.bmm() or torch.matmul()
'''

torch.manual_seed(0)
matrix_left1 = torch.randn(5, 3)
matrix_left2 = torch.randn(5, 3)

print(f"matrix_left1: \n{matrix_left1}")
print(f"matrix_left2: \n{matrix_left2}")

matrix_right1 = torch.randn(3, 4)
matrix_right2 = torch.randn(3, 4)
print(f"matrix_right1: \n{matrix_right1}")
print(f"matrix_right2: \n{matrix_right2}")

print("Let's assume that we have batch of matrix, which is stack of these two matices")
matrix_left = torch.stack([matrix_left1, matrix_left2])
matrix_right = torch.stack([matrix_right1, matrix_right2])

print(f"matrix_left: \n{matrix_left} \n which is shape of {matrix_left.shape}")
print(f"matrix_right: \n{matrix_right}\n which is shape of {matrix_right.shape}")


'''
Exhaustive method: using torch.mm() only with for loop (This is SLOW when matrix gets much larger)
'''

mm_forloop_output = []
for sample_index in range(matrix_left.shape[0]):
  mat_left = matrix_left[sample_index]
  mat_right = matrix_right[sample_index]

  mm_result = torch.mm(mat_left, mat_right)
  mm_forloop_output.append(mm_result)

mm_forloop_stack = torch.stack(mm_forloop_output)
print(f"mat_mul_stack: \n{mm_forloop_stack}\n which is shape of {mm_forloop_stack.shape}")


'''
Good method: using torch.bmm()
'''

mat_mul_bmm = torch.bmm(matrix_left, matrix_right)
print(f"mat_mul_bmm: \n{mat_mul_bmm}\n which is shape of {mat_mul_bmm.shape}")


matrix_left1: 
tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820],
        [-0.8567,  1.1006, -1.0712]])
matrix_left2: 
tensor([[ 0.1227, -0.5663,  0.3731],
        [-0.8920, -1.5091,  0.3704],
        [ 1.4565,  0.9398,  0.7748],
        [ 0.1919,  1.2638, -1.2904],
        [-0.7911, -0.0209, -0.7185]])
matrix_right1: 
tensor([[ 0.5186, -1.3125,  0.1920,  0.5428],
        [-2.2188,  0.2590, -1.0297, -0.5008],
        [ 0.2734, -0.9181, -0.0404,  0.2881]])
matrix_right2: 
tensor([[-0.0075, -0.9145, -1.0886, -0.2666],
        [ 0.1894, -0.2190,  2.0576, -0.0354],
        [ 0.0627, -0.7663,  1.0993,  2.7565]])
Let's assume that we have batch of matrix, which is stack of these two matices
matrix_left: 
tensor([[[ 1.5410, -0.2934, -2.1788],
         [ 0.5684, -1.0845, -1.3986],
         [ 0.4033,  0.8380, -0.7193],
         [-0.4033, -0.5966,  0.1820],
         [-0.8567,  1.1006, -1.0712]],

   

### 2-1 Get attention score in batch (4 pts)
- Now keys and query exist in batch

In [13]:
def get_attention_score_for_a_batch_query(keys, query):
  '''
  This function returns a batch of attention score for each vector in (multi-batch) keys for a given (single-batch) query.
  You can regard 'keys' as hidden states over timestep of Encoder, while query is a hidden state of specific time step of Decoder
  Name 'keys' are used because it is used for calculating attention score (match rate between given vector and query).

  For every C-dimensional vector key, the attention score is a dot product between the key and the query vector.

  Arguments:
    keys (torch.Tensor): Has a shape of [N, T, C]. These are vectors that a query wants attend to
    query (torch.Tensor): Has a shape of [N, C]. This is a vector that attends to other set of vectors (keys and values)

  Output:
    attention_score (torch.Tensor): The attention score in real number that represent how much does query have to attend to each vector in keys
                                    Has a shape of [N, T]

    attention_score[n, i] has to be a dot product value between keys[n, i] and query[n]

  TODO: Complete this function without using for loop
  Hint: Use torch.bmm or torch.matmul after make two input tensors as 3-dim tensors.

  '''

  expanded_query = query.unsqueeze(2) # [N, C] -> [N, C, 1]
  attention_score_matrix = torch.bmm(keys, expanded_query) # [N, T, C] x [N, C, 1] = [N, T, 1]
  attention_score = attention_score_matrix.squeeze(2) # [N, T, 1] -> [N, T]

  return attention_score

torch.manual_seed(0)
num_b = 6
num_t = 23
h_size = 16

keys = torch.randn(num_b,num_t, h_size)
query = torch.randn(num_b, h_size)
out = get_attention_score_for_a_batch_query(keys, query)


In [14]:
'''
Test cases
'''

answer = torch.tensor([ -2.7744,   1.3793,   5.0969,   0.7559,   2.5898,  -0.9475,  -1.1960,
           5.4975,   0.4018,   5.9949,  -5.9428,  -0.4441,   0.6729,  -0.8326,
           3.7091,   1.4913,   2.2062,  -0.2244,  -4.0612,   2.9037,  10.6111,
           4.1383,  -4.6549])

assert out.ndim == 2 and out.shape == torch.Size([num_b, num_t]), "Error: Check output shape"
assert torch.allclose(out[2], answer, atol=1e-4), "Error: The output value is different"
print("Passed all the cases!")

Passed all the cases!


### 2-2 Get attention score in batch (4 pts)
- Implement the same function but in batchified queries

In [15]:
def get_attention_score_for_a_batch_multiple_query(keys, queries):
  '''
  Now you have to implement the attention score for not only single query, but multiple queries.

  This function returns a batch of attention score for each vector in keys for given queries.
  You can regard 'keys' as hidden states over timestep of Encoder, while querys are hidden states over timestep of Decoder
  Name 'keys' are used because it is used for calculating attention score (match rate between given vector and query).

  For every C-dimensional vector key, the attention score is a dot product between the key and the query vector.

  Arguments:
    keys (torch.Tensor): Has a shape of [N, Ts, C]. These are vectors that a query wants attend to
    queries (torch.Tensor): Has a shape of [N, Tt, C]. This is a vector that attends to other set of vectors (keys and values)

  Output:
    attention_score (torch.Tensor): The attention score in real number that represent how much does query have to attend to each vector in keys
                                    Has a shape of [N, Tt, Ts]

    attention_score[n, t, i] has to be a dot product value between keys[n, i] and query[n, t]

  TODO: Complete this function without using for loop
  HINT: Use torch.bmm() with proper transpose (permutation) of given tensors. (You can use atensor.permute())
        Think about which dimension (axis) of tensors has to be multiplied together and resolved (disappear) after matrix multiplication,
        and how the result tensor has to look like (shape)
  '''
  permuted_keys = keys.permute(0, 2, 1) # [N, Ts, C] -> [N, C, Ts]
  attention_score = torch.bmm(queries, permuted_keys) # [N, Tt, C] x [N, C, Ts] = [N, Tt, Ts]

  return attention_score

torch.manual_seed(0)
num_b = 6
num_ts = 23
num_tt = 14
h_size = 16

keys = torch.randn(num_b, num_ts, h_size)
queries = torch.randn(num_b, num_tt, h_size)
att_score = get_attention_score_for_a_batch_multiple_query(keys, queries)

att_score

tensor([[[  6.1462,  -2.0761,  -3.7923,  ...,   2.9819,  -5.4682,  -1.0037],
         [  2.7917,   1.5621,   0.6755,  ...,  12.6860,  -2.9139,   1.1092],
         [  3.2981,  -4.6314,  -2.5517,  ...,   6.7435,  -0.3054,   1.3248],
         ...,
         [ -1.7558,  -2.9616,  -6.6489,  ...,   3.5522,   6.4960,   2.8827],
         [ -1.9945,   5.0151,   2.2012,  ...,  -7.0258,  -1.4581,   3.8804],
         [  1.7817,  -0.5098,   0.5882,  ...,   2.3800, -12.5525,  -5.4968]],

        [[ -2.9907, -15.4518,   4.2723,  ...,   1.3169,  -7.9481,   1.8112],
         [ -0.1470,   2.2430,  -0.6171,  ...,  -1.1141,   5.0494,  -3.9315],
         [ -0.1703,   4.9486,   2.6252,  ...,  -1.6058,   0.7725,   1.6521],
         ...,
         [  2.4992,   4.4271,  -2.2281,  ...,   1.9466,   0.5016,  -0.2215],
         [ -1.8304,  -4.3865,  -2.5648,  ...,   2.6665,  -3.3123,  -0.1541],
         [  1.1768,  -9.2907,  -4.1481,  ...,  -4.1625,  -7.8802,  -6.4050]],

        [[ -3.0427,  -5.7212,   1.0249,  ...

In [16]:
'''
Test cases
'''
answer = torch.Tensor([ 4.9620, -9.6091, -4.9472,  1.4543, -5.6273,  9.1436,  1.4172,  0.0464,
        -5.7033,  4.5473,  7.7498,  1.3405, -3.1877,  2.8759])
answer2 = torch.Tensor([[ 2.5171,  0.6216,  3.7929,  2.6163,  5.3290,  0.3592,  2.3067, -0.1099,
         1.8963,  0.4175, -1.4283,  1.4388, -2.7825, -1.3690, -1.9615, -1.9514,
        -6.4635,  1.9574,  0.1868,  8.5354,  4.6053,  2.8786, -2.1453]])
assert att_score.ndim == 3 and att_score.shape == torch.Size([num_b, num_tt, num_ts]), 'Check the output shape'
assert torch.allclose(att_score[2,:,4], answer, atol=1e-4), 'Calculated result is wrong'
assert torch.allclose(att_score[3,2,:], answer2, atol=1e-4),  'Calculated result is wrong'

print("Passed all the cases!")

Passed all the cases!


### 2-3 Get Masked Softmax (4 pts)
- Implement masked softmax

In [17]:
def get_masked_softmax(attention_score, mask):
  '''
  During the batch computation, each sequence in the batch can have different length.
  To group them as in a single tensor, we usually pad values

  Arguments:
    attention_score (torch.Tensor): The attention score in real number that represent how much does query have to attend to each vector in keys
                                    Has a shape of [N, Tt, Ts]
    mask (torch.Tensor): Boolean tensor with a shape of [N, Ts] that represents whether the corresponding is valid or not.
                         mask[n, i] == 1 if and only if input_batch[n,i] is not a padded value.
                         If input_batch[n,i] is a padded value, then mask[n,i] == 0

  Output:
    attention_weight (torch.Tensor): The attention weight in real number between 0 and 1. The sum of attention_weight along keys timestep dimension is 1.
                                    Has a shape of [N, Tt, Ts]

    attention_weight[n, t, i] has to be an attention weight of values[n, i] for queries[n, t]

  TODO: Complete this function without using for loop
  Hint: You can give -infinity value by -float("inf")
  Caution: Do not directly replace the "attention_score" tensor. Use atensor.clone()
  '''

  masked_attention_score = attention_score.clone()
  expanded_mask = mask.unsqueeze(1) # [N, Ts] -> [N, 1, Ts]
  masked_attention_score.masked_fill_(expanded_mask == 0, -float("inf"))
  attention_weight = torch.nn.functional.softmax(masked_attention_score, dim=-1)

  return attention_weight


'''
Don't change this codes
'''
mask = torch.ones_like(att_score)[:, 0]
mask[4, 15:] = 0
mask[5, 17:] = 0
att_score_modified = att_score.clone()
att_score_modified[4, 15:] = 0
attention_weight = get_masked_softmax(att_score, mask)
attention_weight_for_modified = get_masked_softmax(att_score_modified, mask)
attention_weight, attention_weight_for_modified

(tensor([[[8.5095e-02, 2.2856e-05, 4.1080e-06,  ..., 3.5946e-03,
           7.6878e-07, 6.6794e-05],
          [5.0265e-05, 1.4698e-05, 6.0563e-06,  ..., 9.9608e-01,
           1.6724e-07, 9.3445e-06],
          [2.6409e-02, 9.5063e-06, 7.6066e-05,  ..., 8.2803e-01,
           7.1906e-04, 3.6707e-03],
          ...,
          [1.8632e-04, 5.5794e-05, 1.3970e-06,  ..., 3.7625e-02,
           7.1448e-01, 1.9264e-02],
          [4.1884e-05, 4.6370e-02, 2.7809e-03,  ..., 2.7350e-07,
           7.1609e-05, 1.4910e-02],
          [1.4554e-03, 1.4716e-04, 4.4123e-04,  ..., 2.6474e-03,
           8.6639e-10, 1.0046e-06]],
 
         [[1.4556e-06, 5.6399e-12, 2.0765e-03,  ..., 1.0810e-04,
           1.0234e-08, 1.7722e-04],
          [7.2275e-05, 7.8883e-04, 4.5171e-05,  ..., 2.7478e-05,
           1.3054e-02, 1.6422e-06],
          [8.3620e-06, 1.3978e-03, 1.3690e-04,  ..., 1.9901e-06,
           2.1468e-05, 5.1737e-05],
          ...,
          [7.2220e-02, 4.9651e-01, 6.3913e-04,  ..., 4.155

In [18]:
answer = torch.Tensor([0.0120,     0.0002,     0.0901,     0.0003,     0.0259,     0.0036,
            0.5617,     0.0108,     0.2508,     0.0054,     0.0001,     0.0010,
            0.0000,     0.0005,     0.0375,     0.0000,     0.0000,     0.0000,
            0.0000,     0.0000,     0.0000,     0.0000,     0.0000])
assert torch.allclose(attention_weight[4,3], answer, atol=1e-4), 'Calculated result is wrong'
assert torch.allclose(attention_weight.sum(2),  torch.tensor([1.0]) , atol=1e-6 ), 'Sum of attention weight has to be 1'
assert torch.allclose(attention_weight, attention_weight_for_modified), "Output is different even though only masked part is different"
print("Passed all the cases!")

Passed all the cases!


### 2-4 Implement weighted sum in batchified version (4 pts)

> Add blockquote




In [19]:
def get_batch_weighted_sum(values, attention_weight):
  '''
  This function converts attention score to attention weight

  Argument:
    values (torch.Tensor): Has a shape of [N, Ts, C]. These are vectors that are used to form attention vector
    attention_weight: Has a shape of [N, Ts, Tt], which represents the weight for each vector to compose the attention vector
                      attention_weight[n, s, t] represents weight for value[n, s] that corresponds to a given query, queries[n, t]

  Output:
    attention_vector (torch.Tensor): Weighted sum of values using the attention weight.
                                     Has a shape of [N, Tt, C]

  TODO: Complete this function using torch.bmm
  '''
  attention_vector = torch.bmm(attention_weight, values) # [N, Tt, Ts] x [N, Ts, C] = [N, Tt, C]

  return attention_vector

att_out = get_batch_weighted_sum(keys, attention_weight)
att_out

tensor([[[-3.3373e-01, -2.0938e+00, -4.5334e-02,  ...,  9.5458e-01,
           1.0229e+00,  1.1447e+00],
         [-1.3264e-01, -5.9684e-02,  4.0668e-01,  ..., -2.2277e+00,
           1.4654e+00, -1.2140e+00],
         [-2.1483e-01, -3.9624e-02,  3.4638e-01,  ..., -1.8784e+00,
           1.3381e+00, -1.0378e+00],
         ...,
         [ 4.7269e-01,  2.7082e+00, -1.5966e-01,  ..., -8.5101e-01,
           1.6295e+00, -6.9686e-01],
         [ 1.3049e+00, -7.9595e-01,  5.8837e-01,  ..., -4.2750e-01,
           9.4185e-02, -6.0422e-01],
         [-2.1447e-01,  4.5967e-01,  9.0154e-01,  ..., -1.1010e+00,
           1.5383e+00, -6.2885e-01]],

        [[-2.0093e-02,  9.5785e-01, -1.0546e+00,  ..., -5.6727e-01,
           9.5331e-01, -1.4754e+00],
         [ 2.6579e-01, -9.2264e-01,  8.4929e-01,  ...,  2.4586e+00,
          -2.5894e+00,  2.3314e+00],
         [ 5.3756e-01, -9.0355e-01,  1.7743e-01,  ...,  2.2658e+00,
           3.0777e-01,  2.0951e+00],
         ...,
         [ 1.3244e-01,  8

In [20]:
'''
Test cases
'''
answer = torch.Tensor([-0.9348, -1.2628, -0.9189, -0.3434, -1.6476,  0.1031, -0.6963, -0.7462,
         0.1484,  0.6810,  0.7950,  1.0277, -1.5988,  0.4232, -1.5540,  0.1801])
answer2 = torch.Tensor([-0.9204, -0.9710,  0.3062, -1.0122,  1.1933,  0.1302, -1.0280,  0.0095,
         0.6124,  0.0615, -1.2312, -0.6714, -0.1764, -0.1254])
assert att_out.ndim == 3 and att_out.shape == torch.Size([num_b, num_tt, h_size]), 'Check the output shape'
assert torch.max(torch.abs(att_out[2, 5] - answer)) < 1e-4, 'Calculated result is wrong'
assert torch.max(torch.abs(att_out[3,:,2] - answer2)) < 1e-4,  'Calculated result is wrong'

print("Passed all the cases!")

Passed all the cases!


## Problem 3: Make seq2seq with attention (14 pts)
- Using Pre-defined `TranslatorBi` class, complete a new `TranslatorAtt` class
- If you implement it correctly, you can translate a sentence and get the corresponding attention map

### 3-0 Prepare dataset and tokenizer
- To use the pretrained model correctly, you can use the pretrained vocabularies

In [21]:
'''
Download dataset (originally from NIA AI-Hub)
'''

!gdown 13CGLEULYccogSLByHXPAxSveLZTtnj8c
!unzip -q nia_korean_english_csv.zip

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=13CGLEULYccogSLByHXPAxSveLZTtnj8c

but Gdown can't. Please check connections and permissions.
unzip:  cannot find or open nia_korean_english_csv.zip, nia_korean_english_csv.zip.zip or nia_korean_english_csv.zip.ZIP.


In [21]:
!ls -l /content/

total 318540
-rw-r--r-- 1 root root     13767 Jun 27 11:42 assignment4_pre_defined.py
-rw-r--r-- 1 root root 105906176 Jun 27 11:43 assignment4_values.pt
-rw-r--r-- 1 root root 111149056 Jun 27 11:43 kor_eng_translator_attention_model_best.pt
-rw-r--r-- 1 root root 109051904 Jun 27 11:43 nia_korean_english_csv.zip
-rw-r--r-- 1 root root     41934 Jun 27 11:42 NLP_Assignment_4.py
drwxr-xr-x 1 root root      4096 Jun 25 13:36 sample_data


In [22]:
!unzip -q nia_korean_english_csv.zip

- You can train your own tokenizer with the following code
- Otherwise, you can skip to the next cell to use the pretrained tokenizer

In [ ]:
'''
Caution: Training your own tokenizer will make mismatch between the pretrained model weights and your tokenizer.
This will cause the below assertion not working.
So use this only for testing your own tokenizer.
'''


'''
!gdown 1GMFWREWBVcD5vJdwNFadHmzVStclcyKf
!unzip -q nia-aihub-korean-english-txt.zip

from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)

data_list = list(Path('nia_korean_english').glob('*.xlsx'))

corpus_file   =  [str(path.parent / (path.stem + '_kor.txt')) for path in data_list]
en_corpus_file   =  [str(path.parent / (path.stem + '_eng.txt')) for path in data_list]

vocab_size    = 32000  # Number of maximum size of the vocabulary
limit_alphabet= 6000
output_dir    = Path('hugging_kor_%d'%(vocab_size))
en_output_dir = Path('hugging_eng_%d'%(vocab_size))
output_dir.mkdir(exist_ok=True)
en_output_dir.mkdir(exist_ok=True)
min_frequency = 5

tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet,
               show_progress=True)

tokenizer.save_model(str(output_dir))

en_tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)
en_tokenizer.train(files=en_corpus_file,
                vocab_size=vocab_size,
                min_frequency=min_frequency,
                limit_alphabet=limit_alphabet,
                show_progress=True)
en_tokenizer.save_model(str(en_output_dir))
'''

"\n!gdown 1GMFWREWBVcD5vJdwNFadHmzVStclcyKf\n!unzip -q nia-aihub-korean-english-txt.zip\n\nfrom tokenizers import BertWordPieceTokenizer\ntokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)\n\ndata_list = list(Path('nia_korean_english').glob('*.xlsx'))\n\ncorpus_file   =  [str(path.parent / (path.stem + '_kor.txt')) for path in data_list]\nen_corpus_file   =  [str(path.parent / (path.stem + '_eng.txt')) for path in data_list]\n\nvocab_size    = 32000  # Number of maximum size of the vocabulary\nlimit_alphabet= 6000\noutput_dir    = Path('hugging_kor_%d'%(vocab_size))\nen_output_dir = Path('hugging_eng_%d'%(vocab_size))\noutput_dir.mkdir(exist_ok=True)\nen_output_dir.mkdir(exist_ok=True)\nmin_frequency = 5\n\ntokenizer.train(files=corpus_file,\n               vocab_size=vocab_size,\n               min_frequency=min_frequency,\n               limit_alphabet=limit_alphabet,\n               show_progress=True)\n\ntokenizer.save_model(str(output_dir))\n\nen_tokenizer = 

In [23]:
# Load data and tokenizer

df = pd.read_csv('nia_korean_english.csv')

src_tokenizer = BertTokenizerFast.from_pretrained('hugging_kor_32000',
                                                       strip_accents=False,
                                                       lowercase=False)
tgt_tokenizer = BertTokenizerFast.from_pretrained('hugging_eng_32000',
                                                       strip_accents=False,
                                                       lowercase=False)


# Test tokenizer
print(src_tokenizer.tokenize("안녕하세요. 서강대학교에 오신 걸 환영합니다."))
print(tgt_tokenizer.tokenize("Hello. Welcome to Sogang University."))


['안녕하세요', '.', '서강', '##대학교', '##에', '오신', '걸', '환영', '##합니다', '.']
['hello', '.', 'welcome', 'to', 'so', '##gang', 'university', '.']


In [24]:
class TranslationSet:
  def __init__(self, df, src_tokenizer, tgt_tokenizer):
    self.data = df
    self.src_tokenizer = src_tokenizer
    self.tgt_tokenizer = tgt_tokenizer

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    selected_row = self.data.iloc[idx]
    source = selected_row['원문']
    target = selected_row['번역문']

    source_enc = self.src_tokenizer(source)['input_ids']
    target_enc = self.tgt_tokenizer(target)['input_ids']

    return torch.LongTensor(source_enc), torch.LongTensor(target_enc[:-1]), torch.LongTensor(target_enc[1:])

entireset = TranslationSet(df, src_tokenizer, tgt_tokenizer)
trainset, validset, testset = torch.utils.data.random_split(entireset, [int(len(entireset)*0.9), int(len(entireset)*0.05), len(entireset)-int(len(entireset)*0.9)-int(len(entireset)*0.05)], generator=torch.Generator().manual_seed(42))
# trainset, validset, testset = torch.utils.data.random_split(entireset, [360000, 20000, 20000], generator=torch.Generator().manual_seed(42))

print(f'Dataset Item Example: {entireset[0]}')
print(f'Length of split : Train {len(trainset)}, Valid {len(validset)}, Test {len(testset)}')

def pack_collate(raw_batch):
  source, target, shifted_target = zip(*raw_batch)
  return pack_sequence(source, enforce_sorted=False), pack_sequence(target, enforce_sorted=False), pack_sequence(shifted_target, enforce_sorted=False)

single_loader = DataLoader(trainset, batch_size=1, collate_fn=pack_collate, shuffle=True, num_workers=4, pin_memory=True)
train_loader = DataLoader(trainset, batch_size=64, collate_fn=pack_collate, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(validset, batch_size=128, collate_fn=pack_collate, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(testset, batch_size=128, collate_fn=pack_collate, shuffle=False, num_workers=0, pin_memory=True)

batch = next(iter(train_loader))
batch

Dataset Item Example: (tensor([    2,    11,    70,  4665,  5209, 13306,    71, 12901,  9565, 12435,
           11,  3546, 14567,  4325,  8934,  8407,  7400,  4154,  3252,  6420,
        12985,  4996,  3397,  6461,    18,     3]), tensor([    2, 26268, 23067,    11,  1056,    69, 23067,  2803,  1067,  5155,
         1117,  1042,  2405,  4024,  5520,  1039,  1023, 26268,    18]), tensor([26268, 23067,    11,  1056,    69, 23067,  2803,  1067,  5155,  1117,
         1042,  2405,  4024,  5520,  1039,  1023, 26268,    18,     3]))
Length of split : Train 1442176, Valid 80120, Test 80122


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[PackedSequence(data=tensor([   2,    2,    2,  ..., 7076,   18,    3]), batch_sizes=tensor([64, 64, 64, 64, 64, 64, 64, 64, 63, 60, 57, 55, 52, 50, 48, 42, 38, 36,
         35, 34, 33, 33, 31, 31, 31, 31, 29, 26, 23, 21, 18, 18, 17, 15, 14, 13,
         11, 10,  8,  8,  7,  6,  6,  6,  6,  5,  4,  3,  3,  3,  2,  2,  1,  1,
          1,  1]), sorted_indices=tensor([55, 42, 60, 26, 37, 29, 20,  7,  1, 33,  6,  9, 56, 23, 47, 57,  0, 49,
         15, 41, 24, 38, 14, 18, 46,  4, 13, 44, 61, 50, 40,  2, 52, 21, 39, 51,
         10, 53, 63, 31, 19, 22,  3, 36,  8, 11, 12, 34, 62,  5, 43, 59, 45, 17,
         54, 30, 32, 28, 25, 16, 35, 58, 27, 48]), unsorted_indices=tensor([16,  8, 31, 42, 25, 49, 10,  7, 44, 11, 36, 45, 46, 26, 22, 18, 59, 53,
         23, 40,  6, 33, 41, 13, 20, 58,  3, 62, 57,  5, 55, 39, 56,  9, 47, 60,
         43,  4, 21, 34, 30, 19,  1, 50, 27, 52, 24, 14, 63, 17, 29, 35, 32, 37,
         54,  0, 12, 15, 61, 51,  2, 28, 48, 38])),
 PackedSequence(data=tensor([   2, 

In [25]:
'''
Pre-defined class
'''
NUM_IMG = 100 # This will define how many images you will download

try: # check whether you are running the notebook on colab
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  from google.colab import drive # if you selected to save it in your Google Drive
  drive.mount('/content/drive')



class Trainer:
  def __init__(self, model, optimizer, loss_fn, train_loader, valid_loader, device, save_dir='./'):
    self.model = model
    self.optimizer = optimizer
    self.loss_fn = loss_fn
    self.train_loader = train_loader
    self.valid_loader = valid_loader

    self.model.to(device)

    self.best_valid_accuracy = 0
    self.device = device

    self.training_loss = []
    self.validation_loss = []
    self.validation_acc = []
    self.save_dir = Path(save_dir)
    self.save_dir.mkdir(exist_ok=True)

  def save_model(self, path='kor_eng_translator_attention_model.pt'):
    save_path = self.save_dir / path
    torch.save({'model':self.model.state_dict(), 'optim':self.optimizer.state_dict()}, save_path)

  def train_by_num_epoch(self, num_epochs):
    for epoch in tqdm(range(num_epochs)):
      self.model.train()
      with tqdm(self.train_loader, leave=False) as pbar:
        for batch in pbar:
          loss_value = self._train_by_single_batch(batch)
          self.training_loss.append(loss_value)
          pbar.set_description(f"Epoch {epoch+1}, Loss {loss_value:.4f}")
      self.model.eval()
      validation_loss, validation_acc = self.validate()
      self.validation_loss.append(validation_loss)
      self.validation_acc.append(validation_acc)

      if validation_acc > self.best_valid_accuracy:
        print(f"Saving the model with best validation accuracy: Epoch {epoch+1}, Acc: {validation_acc:.4f} ")
        self.save_model('kor_eng_translator_attention_model_best.pt')
      else:
        self.save_model('kor_eng_translator_attention_model_last.pt')
      self.best_valid_accuracy = max(validation_acc, self.best_valid_accuracy)


  def _train_by_single_batch(self, batch):
    '''
    This method updates self.model's parameter with a given batch

    batch (tuple): (batch_of_input_text, batch_of_label)

    You have to use variables below:

    self.model (SentimentModel/torch.nn.Module): A neural network model
    self.optimizer (torch.optim.adam.Adam): Adam optimizer that optimizes model's parameter
    self.loss_fn (function): function for calculating BCE loss for a given prediction and target
    self.device (str): 'cuda' or 'cpu'

    output: loss (float): Mean binary cross entropy value for every sample in the training batch
    The model's parameters, optimizer's steps has to be updated inside this method
    '''
    src, tgt, shifted_tgt = batch
    src = src.to(self.device)
    tgt = tgt.to(self.device)
    shifted_tgt = shifted_tgt.to(self.device)

    prob = self.model(src, tgt)

    if isinstance(prob, PackedSequence):
      loss = self.loss_fn(prob.data, shifted_tgt.data)
    else:
      loss = self.loss_fn(prob, shifted_tgt)
    loss.backward()
    self.optimizer.step()
    self.optimizer.zero_grad()

    return loss.item()


  def validate(self, external_loader=None):
    '''
    This method calculates accuracy and loss for given data loader.
    It can be used for validation step, or to get test set result

    input:
      data_loader: If there is no data_loader given, use self.valid_loader as default.


    output:
      validation_loss (float): Mean Binary Cross Entropy value for every sample in validation set
      validation_accuracy (float): Mean Accuracy value for every sample in validation set

    '''

    ### Don't change this part
    if external_loader and isinstance(external_loader, DataLoader):
      loader = external_loader
      print('An arbitrary loader is used instead of Validation loader')
    else:
      loader = self.valid_loader

    self.model.eval()

    validation_loss = 0
    num_correct_guess = 0
    num_data = 0
    with torch.inference_mode():
      for batch in tqdm(loader, leave=False):
        src, tgt, shifted_tgt = batch
        src = src.to(self.device)
        tgt = tgt.to(self.device)
        shifted_tgt = shifted_tgt.to(self.device)

        prob = self.model(src, tgt)

        if isinstance(prob, PackedSequence):
          loss = self.loss_fn(prob.data, shifted_tgt.data)
        else:
          loss = self.loss_fn(prob, shifted_tgt)

        validation_loss += loss.item() * len(prob.data)
        if isinstance(prob, PackedSequence):
          num_correct_guess += (prob.data.argmax(dim=-1) == shifted_tgt.data).sum().item()
        else:
          num_correct_guess += (prob.argmax(dim=-1) == shifted_tgt.data).sum().item()
        num_data += len(prob.data)
    return validation_loss / num_data, num_correct_guess / num_data


def nll_loss(pred, target, eps=1e-8):
  '''
  for PackedSequence, the input is 2D tensor

  predicted_prob_distribution has a shape of [num_entire_tokens_in_the_batch x vocab_size]
  indices_of_correct_token has a shape of [num_entire_tokens_in_the_batch]
  '''

  if pred.ndim == 3:
    pred = pred.flatten(0, 1)
  if target.ndim == 2:
    target = target.flatten(0, 1)
  assert pred.ndim == 2
  assert target.ndim == 1
  return -torch.log(pred[torch.arange(len(target)), target] + eps).mean()


Mounted at /content/drive


In [26]:
'''
Pre-defined class

You don't need to change this code
'''
class TranslatorBi(nn.Module):
  def __init__(self, src_tokenizer, tgt_tokenizer, hidden_size=256, num_layers=3):
    super().__init__()
    self.src_tokenizer = src_tokenizer
    self.tgt_tokenizer = tgt_tokenizer

    self.src_vocab_size = self.src_tokenizer.vocab_size
    self.tgt_vocab_size = self.tgt_tokenizer.vocab_size

    self.src_embedder = nn.Embedding(self.src_vocab_size, hidden_size)
    self.tgt_embedder = nn.Embedding(self.tgt_vocab_size, hidden_size)

    self.encoder = nn.GRU(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True)
    self.decoder = nn.GRU(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

    self.decoder_proj = nn.Linear(hidden_size, self.tgt_vocab_size)

  def run_encoder(self, x):
    if isinstance(x, PackedSequence):
      emb_x = PackedSequence(self.src_embedder(x.data), batch_sizes=x.batch_sizes, sorted_indices=x.sorted_indices, unsorted_indices=x.unsorted_indices)
    else:
      emb_x = self.src_embedder(x)

    enc_hidden_state_by_t, last_hidden = self.encoder(emb_x)

    # Because we use bi-directional GRU, there are (num_layers * 2) last hidden states
    # Here, we make it to (num_layers) last hidden states by taking mean of [left-to-right-GRU] and [right-to-left-GRU]
    last_hidden_sum = last_hidden.reshape(self.encoder.num_layers, 2, last_hidden.shape[1], -1).mean(dim=1)
    if isinstance(x, PackedSequence):
      hidden_mean = enc_hidden_state_by_t.data.reshape(-1, 2, last_hidden_sum.shape[-1]).mean(1)
      enc_hidden_state_by_t = PackedSequence(hidden_mean, x[1], x[2], x[3])
    else:
      enc_hidden_state_by_t = enc_hidden_state_by_t.reshape(x.shape[0], x.shape[1], 2, -1).mean(dim=2)


    return enc_hidden_state_by_t, last_hidden_sum

  def run_decoder(self, y, last_hidden_state):
    if isinstance(y, PackedSequence):
      emb_y = PackedSequence(self.tgt_embedder(y.data), batch_sizes=y.batch_sizes, sorted_indices=y.sorted_indices, unsorted_indices=y.unsorted_indices)
    else:
      emb_y = self.tgt_embedder(y)
    out, decoder_last_hidden = self.decoder(emb_y, last_hidden_state)
    return out, decoder_last_hidden

  def forward(self, x, y):
    '''
    x (torch.Tensor or PackedSequence): Batch of source sentences
    y (torch.Tensor or PackedSequence): Batch of target sentences
    '''

    enc_hidden_state_by_t, last_hidden_sum = self.run_encoder(x)
    out, decoder_last_hidden = self.run_decoder(y, last_hidden_sum)

    if isinstance(out, PackedSequence):
      logits = self.decoder_proj(out.data)
      probs = torch.softmax(logits, dim=-1)
      probs = PackedSequence(probs, batch_sizes=y.batch_sizes, sorted_indices=y.sorted_indices, unsorted_indices=y.unsorted_indices)
    else:
      logits = self.decoder_proj(out)
      probs = torch.softmax(logits, dim=-1)
    return probs



### Problem 3.1: Complete the Seq2Seq with Attention (8 pts)
- **Caution**: You have to concatenate [decoder_hidden_state; attention_out] for this implementation
    - You can use different order of concatenation, but the pre-trained model used that specific order, so please follow it so that you can use the pre-trained weight correctly

In [27]:
class TranslatorAtt(TranslatorBi):
  def __init__(self, src_tokenizer, tgt_tokenizer, hidden_size=512, num_layers=3):
    super().__init__(src_tokenizer, tgt_tokenizer, hidden_size, num_layers)

    # define new self.decoder_proj
    self.decoder_proj = nn.Linear(hidden_size * 2, self.tgt_vocab_size)

  def get_attention_vector(self, encoder_hidden_states, decoder_hidden_states, mask):
    '''
    Arguments:
      encoder_hidden_states (torch.Tensor or PackedSequence): Hidden states of encoder GRU. Shape: [N, Ts, C]
      decoder_hidden_states (torch.Tensor or PackedSequence): Hidden states of decoder GRU. Shape: [N, Tt, C]
      mask (torch.Tensor): Masking tensor. If the mask value is 0, the attention weight has to be zero. Shape: [N, Ts]

    Outputs:
      attention_vectors (torch.Tensor or PackedSequence): Attention vectors that has the same shape as decoder_hidden_states
      attention_weights (torch.Tensor): Zero-padded attention weights.
                                You don't need to return it during the training, but it will help you to implement later problem

    TODO: Complete this function using following functions
      get_attention_score_for_a_batch_multiple_query
      get_masked_softmax
      get_batch_weighted_sum
    If the inputs are PackedSequence, the output has to be a PackedSequence
    Use torch.nn.utils.rnn.pad_packed_sequence(packed_sequence, batch_first=True) to convert PackedSequence to Tensor
    Use torch.nn.utils.rnn.pack_padded_sequence(tensor, batch_lens, batch_first=True) to convert Tensor to PackedSequence
    '''
    is_packed = isinstance(encoder_hidden_states, PackedSequence)
    if is_packed:
      encoder_hidden_states, source_lens = pad_packed_sequence(encoder_hidden_states, batch_first=True)
      decoder_hidden_states, target_lens = pad_packed_sequence(decoder_hidden_states, batch_first=True)

    # Write your code from here

    # 1. Calculate attention score using encoder_hidden_states and decoder_hidden_states
    # 2. Mask the attention score using mask and apply softmax to get attention weight
    # 3. Calculate attention vector using attention weight and encoder_hidden_states

 # -------------------------------------------------------------------
    # TODO: 코드를 여기서부터 작성하세요.
    # 1. encoder_hidden_states (keys)와 decoder_hidden_states (queries)를 사용하여 어텐션 스코어를 계산합니다.
    #    (get_attention_score_for_a_batch_multiple_query 함수 사용)
    #    keys: [N, Ts, C], queries: [N, Tt, C] -> attention_score: [N, Tt, Ts]
    attention_scores = get_attention_score_for_a_batch_multiple_query(encoder_hidden_states, decoder_hidden_states)

    # 2. attention_score에 마스크를 적용하고 소프트맥스를 사용하여 어텐션 가중치를 얻습니다.
    #    (get_masked_softmax 함수 사용)
    #    attention_score: [N, Tt, Ts], mask: [N, Ts] -> attention_weights: [N, Tt, Ts]
    attention_weights = get_masked_softmax(attention_scores, mask)

    # 3. attention_weights와 encoder_hidden_states (values)를 사용하여 어텐션 벡터를 계산합니다.
    #    (get_batch_weighted_sum 함수 사용)
    #    values: [N, Ts, C], attention_weights: [N, Tt, Ts] -> attention_vectors: [N, Tt, C]
    #    여기서 values는 encoder_hidden_states가 됩니다.
    attention_vectors = get_batch_weighted_sum(encoder_hidden_states, attention_weights)
    # -------------------------------------------------------------------

    # 출력이 PackedSequence여야 한다면 다시 PackedSequence로 변환합니다.
    # 이 부분은 NLP_Assignment_4.ipynb에 이미 제공된 코드입니다.
    if is_packed:
        # attention_vectors는 [N, Tt, C] 형태이므로, target_lens를 사용하여 패킹합니다.
        attention_vectors = pack_padded_sequence(attention_vectors, target_lens, batch_first=True, enforce_sorted=False)
        # attention_weights는 [N, Tt, Ts] 형태이지만, 문제 조건상 PackedSequence로 반환할 필요는 없다고 되어 있습니다.
        # 하지만 일관성을 위해 PackedSequence 형태로 반환해야 한다면, 이 부분도 pack_padded_sequence를 적용해야 합니다.
        # 현재는 그대로 Tensor 형태로 반환하도록 가정합니다. (혹은 별도의 길이 정보 없이 반환)
        # 만약 PackedSequence로 반환해야 한다면, attention_weights = pack_padded_sequence(attention_weights, target_lens, batch_first=True)
        pass # attention_weights는 그대로 Tensor 형태로 반환 (혹은 필요시 별도 처리)

    return attention_vectors, attention_weights


  def forward(self, x, y):
    '''
    Arguments:
      x (torch.Tensor or PackedSequence): Batch of source sentences
      y (torch.Tensor or PackedSequence): Batch of target sentences
    Output:
      prob_dist (torch.Tensor or PackedSequence): Batch of probability distribution of word for target sentence

    TODO: Complete this function
    '''

    is_packed = isinstance(x, PackedSequence)
    enc_hidden_state_by_t, last_hidden_sum = self.run_encoder(x)
    dec_hidden_state_by_t, decoder_last_hidden = self.run_decoder(y, last_hidden_sum)

    if is_packed:
      mask = pad_packed_sequence(x, batch_first=True)[0] != 0
    else:
      mask = torch.ones(x.shape[0], x.shape[1])

    # TODO: Write your code from here
    # CAUTION:
    #   For the concatenation, you have to concat [dec_hidden_state_by_t; attention_vec], not [attention_vec; dec_hidden_state_by_t]
# -------------------------------------------------------------------
    # TODO: 코드를 여기서부터 작성하세요.

    # 1. get_attention_vector 함수를 호출하여 어텐션 벡터와 어텐션 가중치를 얻습니다.
    # attention_vec: [N, Tt, C] (PackedSequence 또는 Tensor)
    # attention_weight: [N, Tt, Ts] (Tensor)
    attention_vec, attention_weight = self.get_attention_vector(enc_hidden_state_by_t, dec_hidden_state_by_t, mask)

    # PackedSequence일 경우, 디코더 은닉 상태와 어텐션 벡터를 텐서로 변환합니다.
    # self.decoder_proj는 PackedSequence를 직접 처리하지 못하므로, Tensor로 변환해야 합니다.
    # 어텐션 벡터는 get_attention_vector에서 이미 PackedSequence일 경우 처리되므로,
    # 여기서는 decoder_hidden_states만 추가로 패딩 해제합니다.
    if is_packed:
        # dec_hidden_state_by_t가 PackedSequence일 수 있으므로 텐서로 변환합니다.
        # attention_vec은 get_attention_vector 내부에서 PackedSequence로 다시 변환되었을 수 있으므로
        # 이곳에서도 다시 Tensor로 변환하여 concatenate 준비합니다.
        dec_hidden_state_by_t_unpacked, _ = pad_packed_sequence(dec_hidden_state_by_t, batch_first=True)
        attention_vec_unpacked, original_target_lens = pad_packed_sequence(attention_vec, batch_first=True)
        # 중요: original_target_lens는 y의 실제 길이를 나타냅니다.
        # 이 길이는 최종 prob_dist를 다시 PackedSequence로 패킹할 때 사용됩니다.
    else:
        # is_packed가 False인 경우, dec_hidden_state_by_t와 attention_vec은 이미 Tensor입니다.
        dec_hidden_state_by_t_unpacked = dec_hidden_state_by_t
        attention_vec_unpacked = attention_vec
        # Tensor 입력의 경우, 길이는 해당 텐서의 실제 길이가 됩니다.
        # 여기서는 단순히 y의 두 번째 차원(시퀀스 길이)을 사용합니다.
        original_target_lens = torch.full((y.shape[0],), y.shape[1], dtype=torch.long, device=y.device)


    # 2. 디코더 은닉 상태와 어텐션 벡터를 연결합니다. (Concatenation)
    # CAUTION: [dec_hidden_state_by_t; attention_vec] 순서로 연결해야 합니다.
    # 두 텐서의 모양은 [N, Tt, C]로 동일합니다.
    # torch.cat(tensors, dim)은 지정된 차원을 따라 텐서들을 연결합니다.
    # 여기서는 마지막 차원(C)을 따라 연결하여 [N, Tt, 2*C] 형태를 만듭니다.
    combined_output = torch.cat((dec_hidden_state_by_t_unpacked, attention_vec_unpacked), dim=-1) # [N, Tt, C*2]

    # 3. 연결된 벡터를 self.decoder_proj (최종 출력 계층)에 통과시켜 확률 분포를 얻습니다.
    # self.decoder_proj는 [N, Tt, C*2]를 받아 [N, Tt, tgt_vocab_size]를 반환합니다.
    prob_dist = self.decoder_proj(combined_output)

    # 4. PackedSequence였을 경우 다시 PackedSequence로 패킹합니다.
    # 이 부분은 NLP_Assignment_4.ipynb에 이미 제공된 코드입니다.
    if is_packed:
        # original_target_lens는 `if is_packed:` 블록의 첫 부분에서 `pad_packed_sequence(y, batch_first=True)`
        # 를 호출하여 얻는 것이 가장 정확합니다.
        # `dec_hidden_state_by_t_unpacked, original_target_lens = pad_packed_sequence(y, batch_first=True)`
        # 이전에 `decoder_last_hidden`을 얻을 때 `y`가 PackedSequence로 유지될 수 있으므로,
        # `original_target_lens`를 이 위치에서 다시 정확히 얻는 것이 좋습니다.
        # 또는 `get_attention_vector`에서 반환된 `target_lens`를 사용하는 것도 한 방법입니다.
        # 여기서는 `y`가 `PackedSequence`임을 가정하고 `pad_packed_sequence`를 다시 호출하여 길이를 얻습니다.
        _, original_target_lens = pad_packed_sequence(y, batch_first=True)
        prob_dist = pack_padded_sequence(prob_dist, original_target_lens, batch_first=True, enforce_sorted=False)

    return prob_dist


model = TranslatorAtt(src_tokenizer, tgt_tokenizer, hidden_size=32, num_layers=2)

model(batch[0], batch[1])

PackedSequence(data=tensor([[ 0.0385,  0.0021,  0.1027,  ..., -0.0330, -0.2281,  0.1407],
        [ 0.0527, -0.0065,  0.0762,  ..., -0.0574, -0.1954,  0.1672],
        [ 0.0449, -0.0053,  0.0625,  ..., -0.0347, -0.2093,  0.1956],
        ...,
        [ 0.0399, -0.1248,  0.0502,  ...,  0.0542, -0.2272, -0.0025],
        [ 0.1025, -0.0770,  0.1819,  ..., -0.0435, -0.1288,  0.0122],
        [ 0.0969, -0.0971,  0.1644,  ..., -0.0144, -0.1672, -0.0226]],
       grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([64, 64, 64, 64, 64, 64, 64, 64, 62, 61, 58, 55, 54, 52, 49, 48, 45, 43,
        41, 38, 35, 33, 33, 33, 33, 32, 32, 31, 30, 30, 30, 29, 28, 27, 24, 23,
        19, 18, 17, 16, 14, 13, 11, 11, 10, 10,  9,  7,  7,  5,  5,  5,  5,  4,
         4,  4,  4,  3,  3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  1,  1]), sorted_indices=tensor([42, 55, 26,  7,  1, 37, 60, 49, 18,  0, 33, 23, 29, 24, 57,  6, 46,  9,
        41, 40, 38, 47

#### Test your model
- To evaluate your implementation, you have to load the pretrained weight of the same model.
- If your implementation is correct, the resulting value would be the same

In [33]:
# Download pre-trained model and tensor data
!gdown 1IpqJ6U1fuIUf1b8fuwHPSnp7pMU6Qzl4
!gdown 1ch3LdtNprU5QBteP_UJLHaBP9jasUbc3

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1IpqJ6U1fuIUf1b8fuwHPSnp7pMU6Qzl4

but Gdown can't. Please check connections and permissions.
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from th

In [28]:
!ls -l /content/

total 1375680
-rw-r--r-- 1 root root     13767 Jun 27 11:42 assignment4_pre_defined.py
-rw-r--r-- 1 root root 266807867 Jun 27 11:47 assignment4_values.pt
drwx------ 5 root root      4096 Jun 27 11:49 drive
drwxr-xr-x 2 root root      4096 May 21  2023 hugging_eng_32000
drwxr-xr-x 2 root root      4096 May 21  2023 hugging_kor_32000
-rw-r--r-- 1 root root 491782144 Jun 27 11:49 kor_eng_translator_attention_model_best.pt
-rw-r--r-- 1 root root 459917305 May 21  2023 nia_korean_english.csv
-rw-r--r-- 1 root root 190090968 Jun 27 11:46 nia_korean_english_csv.zip
-rw-r--r-- 1 root root     41934 Jun 27 11:42 NLP_Assignment_4.py
drwxr-xr-x 1 root root      4096 Jun 25 13:36 sample_data


In [40]:
model_path = '/content/kor_eng_translator_attention_model_best.pt'
values_path = '/content/assignment4_values.pt'

# Load pretrained weight
model = TranslatorAtt(src_tokenizer, tgt_tokenizer, 512)

# map_location을 사용하여 모델을 현재 디바이스(CPU 또는 CUDA)로 로드
# 또한, state_dict 로드 시에 strict=False를 추가하여
# 혹시라도 state_dict의 키 이름이 정확히 일치하지 않는 경우를 대비할 수 있습니다.
# (예: 이전 버전과의 호환성 문제)
try:
    state_dict = torch.load(model_path, map_location=device)['model']
    model.load_state_dict(state_dict, strict=True) # strict=True로 정확히 일치하는지 확인
    print(f"Model loaded successfully from {model_path} to {device}.")
except Exception as e:
    print(f"Error loading model from {model_path}: {e}")
    print("Trying to load with strict=False (may ignore some key mismatches)...")
    try:
        state_dict = torch.load(model_path, map_location=device)['model']
        model.load_state_dict(state_dict, strict=False) # strict=False로 다시 시도
        print(f"Model loaded successfully with strict=False from {model_path} to {device}.")
    except Exception as e_non_strict:
        print(f"Error loading model even with strict=False: {e_non_strict}")
        print("Please ensure the model file is not corrupted and matches the model architecture.")

model.to(device) # 모델을 최종적으로 올바른 디바이스로 이동
model.eval() # 모델을 평가 모드로 전환

# Load the pre-calculated example and result
try:
    prob3_values = torch.load(values_path, map_location=device) # values도 디바이스로 로드
    single_batch_example = prob3_values['single_test_batch']
    packed_batch_example = prob3_values['packed_test_batch']
    correct_single_out = prob3_values['correct_single_out']
    correct_packed_out = prob3_values['correct_packed_out']
    print(f"Test values loaded successfully from {values_path} to {device}.")
except Exception as e:
    print(f"Error loading test values from {values_path}: {e}")
    print("Please ensure 'assignment4_values.pt' is not corrupted.")


# 이후의 테스트 코드들 (`single_out = model(single_batch_example[0], single_batch_example[1])` 등)은
# 이 수정된 로드 코드 다음에 위치해야 합니다.

print("\nPretrained weights and test data loading section completed.")


Error loading model from /content/kor_eng_translator_attention_model_best.pt: name 'device' is not defined
Trying to load with strict=False (may ignore some key mismatches)...
Error loading model even with strict=False: name 'device' is not defined
Please ensure the model file is not corrupted and matches the model architecture.


NameError: name 'device' is not defined

In [29]:
# Load pretrained weight
model = TranslatorAtt(src_tokenizer, tgt_tokenizer, 512)
state_dict = torch.load('kor_eng_translator_attention_model_best.pt', map_location='cpu')['model']
model.eval()
model.load_state_dict(state_dict)

# Load the pre-calculated example and result
prob3_values = torch.load('assignment4_values.pt', weights_only=False)
single_batch_example, packed_batch_example, correct_single_out, correct_packed_out = prob3_values['single_test_batch'], prob3_values['packed_test_batch'], prob3_values['correct_single_out'],  prob3_values['correct_packed_out']

In [ ]:
'''
Test Case for Single-size Batch
'''
single_out = model(single_batch_example[0], single_batch_example[1])

assert isinstance(single_out, torch.Tensor), "The output of model for Tensor has to be Tensor"
assert torch.allclose(single_out, correct_single_out, atol=1e-4), "The output value is different from the expected"

AssertionError: The output value is different from the expected

In [ ]:
'''
Test Case for Batch with PackedSequence
'''
packed_out = model(packed_batch_example[0], packed_batch_example[1])

assert isinstance(packed_out, PackedSequence), "The output of model for PackedSequence has to be PackedSequence"
assert (packed_out.batch_sizes == correct_packed_out.batch_sizes).all(), "Output's batch_sizes is wrong"
assert (packed_out.sorted_indices == correct_packed_out.sorted_indices).all(), "Output's sorted_indices is wrong"

assert torch.allclose(packed_out.data, correct_packed_out.data, atol=1e-4),  "The output value is different from the expected"


AssertionError: The output value is different from the expected

### Train the model (Optional)
- You can try to train your model, but you can just load the pretrained data
- You don't have to train the model yourself.

In [30]:
model = TranslatorAtt(src_tokenizer, tgt_tokenizer, 512)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

save_dir = '/drive/MyDrive/NLP_Assignment4/' if IN_COLAB else './'

trainer = Trainer(model, optimizer, nll_loss, train_loader, valid_loader, 'cuda')

In [31]:
# Otherwise, you have to load the model weight
model = TranslatorAtt(src_tokenizer, tgt_tokenizer, 512)
state_dict = torch.load('kor_eng_translator_attention_model_best.pt', map_location='cpu')['model']
model.load_state_dict(state_dict)
model.eval()

TranslatorAtt(
  (src_embedder): Embedding(32000, 512)
  (tgt_embedder): Embedding(32000, 512)
  (encoder): GRU(512, 512, num_layers=3, batch_first=True, bidirectional=True)
  (decoder): GRU(512, 512, num_layers=3, batch_first=True)
  (decoder_proj): Linear(in_features=1024, out_features=32000, bias=True)
)

### Problem 3.2: Implement Inference with Attention Weights (6 pts)
- In this problem, you have to implement an inference code that returns translation for given source sentence, but also **attention weights** between source sentence and target sentence

In [ ]:
def translate(model, source_sentence):
  '''
  This function translates a given sentence using a given model.
  It returns the tokenized source sentence, tokenized translated sentence, translated sentence in string, and attention map

  Arguments:
    model (TranslatorAtt): Translator model with attention
    source_sentence (str): Sentence to translate

  Returns:
    input_tokens (list): Source sentence in a list of token in token_id
    predicted_tokens (list): Translated sentence in a list of token in token_id
    decoded_string (str): Translated sentence in string
    attention_map (torch.Tensor): Attention weight between each token of source sentence and target sentence. Has a shape of [Tt, Ts]

  '''

  input_tokens = model.src_tokenizer.encode(source_sentence)
  input_tensor = torch.LongTensor(input_tokens).unsqueeze(0)
  mask = torch.ones_like(input_tensor)
  enc_hidden_state_by_t, last_hidden_sum = model.run_encoder(input_tensor)

  # Setup for 0th step
  current_hidden = last_hidden_sum
  current_decoder_token = torch.LongTensor([[2]]) # start of sentence token
  total_output = []
  total_attetion_weights = []

  for i in range(100): # You can chage it to while True:
    emb = model.tgt_embedder(current_decoder_token)
    '''
    TODO: Complete the code here

    You have to
      1) run decoder rnn for a single step
      2) get attention weight (variable name: att_weight) and attention vector.
         att_weight.shape == torch.Size([1, 1, len(tokenized_sentence)])
      3) concat decoder out and attention vector
      4) calculate probabilty logit (variable name: logit)
    '''
    dec_output_one_step, last_hidden = model.decoder_gru(emb, current_hidden)
    att_vec, att_weight = model.get_attention_vector(
        encoder_hidden_states_unpacked.unsqueeze(0) if encoder_hidden_states_unpacked.ndim == 2 else encoder_hidden_states_unpacked, # N=1 차원 추가
        dec_output_one_step, # [1, 1, C]
        mask # [1, Ts]
    )
    combined_output = torch.cat((dec_output_one_step, att_vec), dim=-1)
    logit = model.decoder_proj(combined_output) # [1, 1, vocab_size]


    # You don't have to change the codes below.
    # Declare logit and last_hidden properly so that the code below can run without error
    selected_token = torch.argmax(logit, dim=-1)
    current_decoder_token = selected_token
    current_hidden = last_hidden
    if current_decoder_token == 3: ## end of sentence token
      break
    total_output.append(selected_token[0])
    total_attetion_weights.append(att_weight[0,0])
  predicted_tokens = torch.cat(total_output, dim=0).tolist()
  attention_map = torch.stack(total_attetion_weights, dim=1)

  return  input_tokens, predicted_tokens, model.tgt_tokenizer.decode(predicted_tokens), attention_map

model.cpu()
test_sentence = '이 알고리즘을 사용하면 한국어 단어와 영어 단어가 어떻게 연결되는지를 알 수 있습니다.'
input_tokens, pred_tokens, translated_string, att_weights  = translate(model, test_sentence)
print(translated_string)

In [ ]:
'''
Test Case: This has to be run with the provided pre-trained weight, which canbe downloaded by the line below
'''
test_sentence = '이 알고리즘을 사용하면 한국어 단어와 영어 단어가 어떻게 연결되는지를 알 수 있습니다.'
input_tokens, pred_tokens, translated_string, att_weights  = translate(model, test_sentence)

correct_output = 'using this algorithm, you can see how korean words and english words are connected.'
answer = torch.tensor([1.6955e-07, 1.1118e-07, 6.0198e-10, 9.1661e-01, 7.8413e-18, 1.2012e-17,
        7.6764e-29, 1.2549e-20, 5.7431e-19, 3.3563e-31, 1.1225e-21, 1.3192e-27,
        3.6772e-26, 2.2244e-23, 3.6098e-20, 1.5309e-21, 7.5093e-05])
answer2 = torch.tensor([1.1643e-11, 3.9906e-30, 1.2813e-33, 2.7519e-13, 2.3483e-15, 4.2758e-12,
        1.9385e-18, 6.8541e-16, 4.9662e-18, 5.0304e-33, 7.2299e-26, 4.4580e-25,
        3.7096e-23, 7.5614e-22, 4.5226e-22, 2.3576e-25, 1.7577e-12])
answer3 = torch.tensor([1.2012e-17, 1.7528e-21, 1.1316e-18, 2.7204e-17, 8.2384e-10, 3.4510e-11,
        1.8289e-09, 1.1806e-12, 3.9218e-19, 2.8321e-16, 1.3933e-12, 3.6876e-10,
        4.1782e-07, 1.0905e-02, 9.8673e-01, 2.3651e-03, 1.7207e-08, 4.2758e-12])


assert translated_string == correct_output, 'Translated sentence is wrong'
assert att_weights.shape == torch.Size([18, 17]), 'Attention weight has wrong shape'
assert torch.allclose(att_weights[0], answer, rtol=1e-4), 'Calculated result is wrong'
assert torch.allclose(att_weights[-1], answer2, rtol=1e-4), 'Calculated result is wrong'
assert torch.allclose(att_weights[:,5], answer3, rtol=1e-4), 'Calculated result is wrong'

print("Passed all the cases!")


### Plot attention map
- If you completed `translate()`, you can visualize the result of attention weight as below

In [ ]:
plt.figure(figsize=(len(pred_tokens)*0.8, len(input_tokens)*0.8))
x_axis_label = [model.tgt_tokenizer.decode(x) for x in pred_tokens]
y_axis_label = [model.src_tokenizer.decode(x) for x in input_tokens]

plt.imshow(att_weights.detach())
plt.xticks(range(len(x_axis_label)), x_axis_label, fontsize=15,rotation = 45)
plt.yticks(range(len(y_axis_label)), y_axis_label, fontsize=15)
None

## Problem 4: Transformer and Self Attention (14 pts)
- In this problem, you will implement the query-key-value calculation that was used for Transformer
- Also, you have to implement self-attention and cross-attention, which are the core components of Transformer


### 4-1 Implement Query-Key-Value Calculation (2 pts)

In [ ]:
def get_query_key_value(input_tensor, qkv_layer):
  '''
  This function returns key, query, and value that is calculated by input tensor and nn_layer.

  Arguments:
    input_tensor (torch.Tensor): Has a shape of [N, T, C]
    kqv_layer (torch.nn.Linear): Linear layer with in_features=C and out_features=Cn * 3

  Outputs:
    queries (torch.Tensor): Has a shape of [N, T, Cn]
    keys (torch.Tensor): Has a shape of [N, T, Cn]
    values (torch.Tensor): Has a shape of [N, T, Cn]

  TODO: Complete this function without using for loop
  Hint: Use torch.chunk() to split a tensor into given number of chunks
  '''
  return

torch.manual_seed(0)
test = torch.randn(4, 17, 8)
linear = nn.Linear(8, 16 * 3)
queries, keys, values = get_query_key_value(test, linear)

In [ ]:
'''
Test cases
'''
answer = torch.Tensor([ 0.5393,  0.0587,  0.6597, -1.1150, -0.7343,  0.3282,  0.0551,  0.0178,
         0.4408, -0.3078,  0.3289, -0.4874,  0.2256, -0.1007, -0.4304, -0.2109])
answer2 = torch.Tensor([ 0.8704, -0.2256,  0.6611,  0.0332, -0.5233, -0.1159,  0.1805,  0.7238,
         0.5590,  0.7260,  1.3096,  0.2465,  1.1961,  0.1751, -0.9674,  0.6297])
assert keys.ndim == queries.ndim == values.ndim == 3
assert keys.shape == queries.shape == values.shape == torch.Size([4, 17, 16])
assert not (keys==queries).any() and not (keys==values).any() and not (values==queries).any()
assert torch.allclose(queries[2, 13], answer, atol=1e-4)
assert torch.allclose(values[0, 3], answer2, atol=1e-4)

print('Passed all the cases!')

### 4-2 Implement 3d masked softmax (1 pts):
  - This would be almost similar to ``get_masked_softmax()``
  - It is common to use N x Tq x Tk attention score and mask, but we will use N x Tk x Tq

In [ ]:
def get_3d_masked_softmax(attention_score, mask):
  '''
  During the batch computation, each sequence in the batch can have different length.
  To group them as in a single tensor, we usually pad values

  Arguments:
    attention_score (torch.Tensor): The attention score in real number that represent how much does query have to attend to each vector in keys
                                    Has a shape of [N, Tq, Tk]
    mask (torch.Tensor): Boolean tensor with a shape of [N, Tq, Tk] that represents whether the corresponding is valid or not.
                         mask[n, tq, tk] == 1 if and only if input_batch[n,tk] is not a padded value.
                         If input_batch[n,tk] is a padded value, then mask[n,tq, tk] == 0

  Output:
    attention_weight (torch.Tensor): The attention weight in real number between 0 and 1. The sum of attention_weight along keys timestep dimension is 1.
                                    Has a shape of [N, Tq, Tk]

    attention_weight[n, t, i] has to be an attention weight of values[n, i] for queries[n, t]

  TODO: Complete this function without using for loop
  Caution: Do not directly mask the attention score. use .clone() instead

  '''
  assert attention_score.ndim == mask.ndim == 3

  return

'''
Don't change this codes
'''
torch.manual_seed(0)
mask = torch.ones([3, 9, 9])
mask[1, :, 2:] = 0
mask[2, :, 7:] = 0
att_score = torch.randn([3, 9, 9]).transpose(1,2) # Transpose here is just to match the test value for assertion
att_score_modified = att_score.clone()
att_score_modified[1, :, 2:] = 0
attention_weight = get_3d_masked_softmax(att_score, mask)
attention_weight_for_modified = get_3d_masked_softmax(att_score_modified, mask)
attention_weight, attention_weight_for_modified

In [ ]:
'''
Test cases
'''

answer = torch.tensor([0.1348, 0.1429, 0.2938, 0.0369, 0.0748, 0.0577, 0.2591, 0.0000, 0.0000])

assert attention_weight.ndim == 3
assert torch.allclose(attention_weight[2,:, 0], answer, atol=1e-4)
assert not torch.isnan(attention_weight).any(), "Error: There is a nan value in attention_weight"
assert torch.allclose(attention_weight, attention_weight_for_modified, atol=1e-4), "Error: The attention_weight are different even though only masked item is different"

print('Passed all the cases!')

### 4-3 Implement Self-Attention (2 pts)

In [ ]:
def get_self_attention(input_tensor, qkv_layer, mask_2d):
  '''
  This function returns output of self-attention for a given input tensor using with a given kqv_layer

  Arguments:
    input_tensor (torch.Tensor): Has a shape of [N, T, C]
    kqv_layer (torch.nn.Linear): Linear layer with in_features=C and out_features=Cn * 3
    mask (torch.Tensor): Boolean tensor with a shape of [N, T] that represents whether the corresponding is valid or not.

  Outputs:
    output (torch.Tensor): Has a shape of [N, T, Cn]

  TODO: Complete this function using your completed functions of below:
        get_query_key_value()
        get_attention_score_for_a_batch_multiple_query()
        get_3d_masked_softmax()
        get_batch_weighted_sum()
  '''
  mask_3d = mask_2d.unsqueeze(1).repeat(1,input_tensor.shape[1],1)

  return


torch.manual_seed(0)
test = torch.randn(5, 17, 8)
linear = nn.Linear(8, 16 * 3)
mask_2d = torch.ones([5, 17])
mask_2d[2, 4:] = 0
mask_2d[4, 14:] = 0

att_vecs = get_self_attention(test, linear, mask_2d)
modified_test = test.clone()
modified_test[2, 4:] = 0
modified_test[4, 14:] = 0
modified_att_vecs = get_self_attention(modified_test, linear, mask_2d)

In [ ]:
'''
Test cases
'''
answer = torch.Tensor([-0.3925, -0.0043,  0.0343, -0.6713,  0.2388, -0.4703, -0.2195, -0.1550,
        -0.0830, -0.4170, -0.1829,  0.3884,  0.2899,  0.1284,  0.0225, -0.5960])
answer2 = torch.Tensor([-0.4078,  0.0173,  0.2670, -0.7959, -0.0314, -0.3455,  0.5751, -0.5806,
        -0.3328, -0.2571, -0.4913, -0.1833,  0.6236, -0.5167,  0.3256, -0.9818])
assert torch.allclose(att_vecs[3, 2], answer, atol=1e-4)
assert torch.allclose(att_vecs[0, 11], answer2, atol=1e-4)
assert torch.allclose(att_vecs[4, :14], modified_att_vecs[4, :14], atol=1e-6)


print('Passed all the cases!')

### 4-4 Implement Multi-head split and concat (2 pts)

In [ ]:
def get_multihead_split(x, num_head):
  '''
  This function returns split tensor for multi-head attention

  Arguments:
    x (torch.Tensor): Has a shape of [N, T, C]
    num_head (int): Number of heads

  Output:
    x (torch.Tensor): Has a shape of [N * num_head, T, C // num_head]
    The order of N * num_head is [Batch1_head1, Batch1_head2, ..., Batch1_headN, Batch2_head1, Batch2_head2, ..., Batch2_headN, ...]
  '''
  assert x.shape[-1] % num_head == 0

  # TODO: Complete this function
  return

torch.manual_seed(0)
dummy_input = torch.randn(4, 17, 32)
head_split_output = get_multihead_split(dummy_input, 8)
head_split_output.shape

assert head_split_output.shape == torch.Size([32, 17, 4])
assert (dummy_input[0, :, 4:8] == head_split_output[1]).all()
assert (dummy_input[0, 3, 8:12] == head_split_output[2, 3, :]).all()
assert (dummy_input[2, 10, 16:20] == head_split_output[20, 10, :]).all()
print('Passed all the cases!')


In [ ]:
def get_multihead_concat(x, num_head):
  '''
  This function returns concat tensor for multi-head attention

  Arguments:
    x (torch.Tensor): Has a shape of [N * num_head, T, C // num_head]
    num_head (int): Number of heads
  Outputs:
    x (torch.Tensor): Has a shape of [N, T, C]
  '''
  # TODO: Complete this function
  assert x.shape[0] % num_head == 0

  return

head_cat_output = get_multihead_concat(head_split_output, 8)
print(f"Output shape: {head_cat_output.shape}")
assert head_cat_output.shape == torch.Size([4, 17, 32])
assert (dummy_input == head_cat_output).all()
print('Passed all the cases!')

### 4-5 Implement Transformer-like multi-head self-attention (3 pts)

In [ ]:
def get_multi_head_self_attention(input_tensor, qkv_layer, output_proj_layer, mask, num_head=8):
  '''
  This function returns output of multi-headed self-attention for a given input tensor using with a given kqv_layer

  Arguments:
    input_tensor (torch.Tensor): Has a shape of [N, T, C]
    qkv_layer (torch.nn.Linear): Linear layer with in_features=C and out_features=Cn * 3
    output_proj_layer (torch.nn.Linear): Linear layer with in_features=Cn and out_features=C
    mask (torch.Tensor): Boolean tensor with a shape of [N, T, T] that represents whether the corresponding is valid or not.
                         mask[n, t] == 1 if and only if input_batch[n,t] is not a padded value.
                         If input_batch[n,t] is a padded value, then mask[n,t] == 0
    num_head (int): Number of heads

  Outputs:
    output (torch.Tensor): Has a shape of [N, T, Cn]

  TODO: Complete this function using your completed functions of below:
        get_query_key_value(): Get QKV from input_tensor and kqv_layer

        get_multihead_split(): Split QKV into multiple heads

        get_attention_score_for_a_batch_multiple_query(): Get attention score for a batch of multiple queries
          CAUTION: You have to scale the attention score by dividing by sqrt(Cn // num_head)
          HINT: att_score /= keys.shape[-1] ** 0.5

        get_3d_masked_softmax(): Get masked softmax/
          CAUTION: You have to repeat mask for num_head times to use it for multi-head attention
          USE head_repeated_mask
        get_batch_weighted_sum(): Get batch weighted sum

        get_multihead_concat(): Concatenate multiple heads into a single tensor

        Additionally, use output_proj_layer to project concatenated tensor at the final step
  '''
  head_repeated_mask = mask.unsqueeze(1).repeat(1, num_head, 1, 1).reshape(-1, mask.shape[1], mask.shape[2])

  return


torch.manual_seed(0)
test = torch.randn(5, 17, 16)
linear = nn.Linear(16, 16 * 3)
out_proj = nn.Linear(16, 16)

mask = torch.ones([5, 17, 17])
mask[2, :, 4:] = 0
mask[4, :, 14:] = 0

att_vecs = get_multi_head_self_attention(test, linear, out_proj, mask, num_head=4)
att_vecs

In [ ]:
wrong_answer = torch.tensor([ 0.0105,  0.2337,  0.1587, -0.0346,  0.1446,  0.0231,  0.0296, -0.0437,
        -0.1101,  0.2079, -0.0813, -0.2343, -0.3270,  0.2156, -0.0427, -0.0313])
wrong_answer2 = torch.tensor([ 0.0557,  0.1026, -0.0834,  0.1770, -0.4083, -0.1543,  0.2025, -0.1570,
         0.0974, -0.0317,  0.0569, -0.0610,  0.0063,  0.0107, -0.0727,  0.0796])

assert not torch.allclose(att_vecs[0, 0], wrong_answer, atol=1e-4), "Error: You did not include attention score scaling. Read the CAUTION!"
assert not torch.allclose(att_vecs[0, 0], wrong_answer2, atol=1e-4), "Error: You did not include output projection. Read the TODO!"
print('Passed the test cases!')

In [ ]:
'''
Compare with official implementation of PyTorch
'''

official_attention = torch.nn.MultiheadAttention(16, num_heads=4, batch_first=True)
official_attention.in_proj_weight.data = linear.weight.data
official_attention.in_proj_bias.data = linear.bias.data
official_attention.out_proj.weight.data = out_proj.weight.data
official_attention.out_proj.bias.data = out_proj.bias.data

head_repeated_mask = mask.unsqueeze(1).repeat(1, 4, 1, 1).reshape(-1, mask.shape[1], mask.shape[2]).transpose(1,2)
official_attention_output, attention_weights = official_attention(test, test, test, attn_mask=head_repeated_mask==0)

assert torch.allclose(att_vecs, official_attention_output, atol=1e-4), "Your output is different from the official output"
print('Passed the test cases!')

### 4-6 Implement it as a single module (2 pts)

In [ ]:
class SelfAttention(nn.Module):
  def __init__(self, input_size, hidden_size, num_head, mask_value=0):
    super().__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.qkv = nn.Linear(self.input_size, self.hidden_size * 3)
    self.out_proj = nn.Linear(self.hidden_size, self.input_size)
    self.mask_value = mask_value
    self.num_head = num_head
    assert self.hidden_size % self.num_head == 0
    self.dim_per_head = self.hidden_size // self.num_head

  '''
  TODO: Implement this function as functions you implemented above
  '''
  def _get_qkv(self, x):
    return

  def _get_multihead_split(self, x):
    return

  def _get_multiheaded_att_score(self, keys, queries):
    return

  def _get_masked_softmax(self, score, masks):
    return

  def _get_weighted_sum(self, values, weights):
    return

  def forward(self, x, mask=None):
    '''
    TODO: Implement this function using the functions you implemented above
    '''
    if mask is None:
      mask = torch.ones([x.shape[0], x.shape[1], x.shape[1]])
    if mask.ndim == 2:
      # TODO: Convert mask to 3D mask
      pass

    return

torch.manual_seed(0)
attention_module = SelfAttention(512, 512, 8)
test = torch.randn(5, 17, 512)
mask_2d = torch.ones([5, 17])
mask_2d[2, 4:] = 0
mask_2d[4, 14:] = 0

out = attention_module(test, mask_2d)
out.shape

In [ ]:
'''
Test case
'''

official_attention = torch.nn.MultiheadAttention(512, num_heads=8, batch_first=True)
official_attention.in_proj_weight.data = attention_module.qkv.weight.data
official_attention.in_proj_bias.data = attention_module.qkv.bias.data
official_attention.out_proj.weight.data = attention_module.out_proj.weight.data
official_attention.out_proj.bias.data = attention_module.out_proj.bias.data

official_attention_output, attention_weights = official_attention(test, test, test, key_padding_mask=mask_2d==0)

assert torch.allclose(out, official_attention_output, atol=1e-4), "Your output is different from the official output"
print('Passed the test cases!')

### Test your SelfAttention module on Encoder Layer

In [ ]:
class MLP(nn.Module):
  def __init__(self, in_size, hidden_size):
    super().__init__()
    self.input_size = in_size
    self.layer = nn.Sequential(nn.Linear(in_size, hidden_size),
                              nn.ReLU(),
                              nn.Linear(hidden_size, in_size))
  def forward(self, x):
    return self.layer(x)

class PosEncoding(nn.Module):
  def __init__(self, size, max_t):
    super().__init__()
    self.size = size
    self.max_t = max_t
    self.register_buffer('encoding', self._prepare_emb())

  def _prepare_emb(self):
    dim_axis = 10000**(torch.arange(self.size//2) * 2 / self.size)
    timesteps = torch.arange(self.max_t)
    pos_enc_in = timesteps.unsqueeze(1) / dim_axis.unsqueeze(0)
    pos_enc_sin = torch.sin(pos_enc_in)
    pos_enc_cos = torch.cos(pos_enc_in)

    pos_enc = torch.stack([pos_enc_sin, pos_enc_cos], dim=-1).reshape([self.max_t, 512])
    return pos_enc

  def forward(self, x):
    return self.encoding[x]

class ResidualLayerNormModule(nn.Module):
  def __init__(self, submodule):
    super().__init__()
    self.submodule = submodule
    self.layer_norm = nn.LayerNorm(self.submodule.input_size)

  def forward(self, x, mask=None, y=None):
    if y is not None:
      res_x = self.submodule(x, y, mask)
    elif mask is not None:
      res_x = self.submodule(x, mask)
    else:
      res_x = self.submodule(x)
    x =  x + res_x
    return self.layer_norm(x)

class EncoderLayer(nn.Module):
  def __init__(self, in_size, emb_size, mlp_size, num_head):
    super().__init__()
    self.att_block = ResidualLayerNormModule(SelfAttention(in_size, emb_size, num_head))
    self.mlp_block = ResidualLayerNormModule(MLP(emb_size, mlp_size))

  def forward(self, x):
    out = self.mlp_block(self.att_block(x['input'], x['mask']))
    return {'input':out, 'mask':x['mask']}


torch.manual_seed(0)
encoder_layer = EncoderLayer(512, 512, 2048, 8)
test = torch.randn(5, 17, 512)
mask_2d = torch.ones([5, 17])
mask_2d[2, 4:] = 0
mask_2d[4, 14:] = 0

out = encoder_layer({'input':test, 'mask':mask_2d})

In [ ]:
'''
Test case
'''

official_encoder_layer = nn.TransformerEncoderLayer(512, 8, 2048, batch_first=True, dropout=0)

official_encoder_layer.self_attn.in_proj_weight.data = encoder_layer.att_block.submodule.qkv.weight.data
official_encoder_layer.self_attn.in_proj_bias.data = encoder_layer.att_block.submodule.qkv.bias.data
official_encoder_layer.self_attn.out_proj.weight.data = encoder_layer.att_block.submodule.out_proj.weight.data
official_encoder_layer.self_attn.out_proj.bias.data = encoder_layer.att_block.submodule.out_proj.bias.data
official_encoder_layer.linear1.weight.data = encoder_layer.mlp_block.submodule.layer[0].weight.data
official_encoder_layer.linear1.bias.data = encoder_layer.mlp_block.submodule.layer[0].bias.data
official_encoder_layer.linear2.weight.data = encoder_layer.mlp_block.submodule.layer[2].weight.data
official_encoder_layer.linear2.bias.data = encoder_layer.mlp_block.submodule.layer[2].bias.data
official_encoder_layer.norm1.weight.data = encoder_layer.att_block.layer_norm.weight.data
official_encoder_layer.norm1.bias.data = encoder_layer.att_block.layer_norm.bias.data
official_encoder_layer.norm2.weight.data = encoder_layer.mlp_block.layer_norm.weight.data
official_encoder_layer.norm2.bias.data = encoder_layer.mlp_block.layer_norm.bias.data

official_encoder_output = official_encoder_layer(test, src_key_padding_mask=mask_2d==0)

assert torch.allclose(official_encoder_output, out['input'], atol=1e-4), "Your output is different from the official output"
print('Passed the test case!')

### 4-7 Implement Transformer-like multi-head cross-attention (2 pts)
- Implement ``CrossAttention`` inheriting `SelfAttention` class

In [ ]:
class CrossAttention(SelfAttention):
  def __init__(self, input_size, hidden_size, num_head, mask_value=0):
    super().__init__(input_size, hidden_size, num_head, mask_value)

  def forward(self, q_seq, kv_seq, encoder_mask=None):
    '''
    Arguments:
      q_seq (torch.Tensor): Sequence to be used for query
      kv_seq (torch.Tensor): Sequence to be used for key and value
      mask (torch.Tensor): Masking tensor. If the mask value is 0, the attention weight has to be zero. Shape: [N, Ty]

    Outs:
      output (torch.Tensor): Output of cross attention. Shape: [N, Tx, C]

    TODO: Complete this function using your completed functions of below:
    '''
    if encoder_mask is None:
      encoder_mask = torch.ones([q_seq.shape[0], kv_seq.shape[1]]) # Then convert to 3D mask
    if encoder_mask.ndim == 2:
      # TODO: Convert mask to 3D mask
      pass

    return

class DecoderLayer(nn.Module):
  def __init__(self, in_size, emb_size, mlp_size, num_head):
    super().__init__()
    self.att_block = ResidualLayerNormModule(SelfAttention(in_size, emb_size, num_head))
    self.cross_att_block = ResidualLayerNormModule(CrossAttention(in_size, emb_size, num_head))
    self.mlp_block = ResidualLayerNormModule(MLP(emb_size, mlp_size))

  def forward(self, x):
    out = self.att_block(x['input'], x['decoder_mask'])
    out = self.cross_att_block(out,  x['encoder_mask'], x['encoder_out'])
    out = self.mlp_block(out)
    return {'input':out, 'decoder_mask':x['decoder_mask'], 'encoder_out':x['encoder_out'], 'encoder_mask':x['encoder_mask']}

torch.manual_seed(0)

decoder_layer = DecoderLayer(512, 512, 2048, 8)
test_src = torch.randn(5, 17, 512)
test_tgt = torch.randn(5, 19, 512)
mask_src = torch.ones([5, 17, 19])
mask_src[2, 4:] = 0
mask_src[4, 14:] = 0
mask_tgt = torch.tril(torch.ones(test_tgt.shape[0], test_tgt.shape[1], test_tgt.shape[1]))

out = decoder_layer({'input':test_tgt, 'decoder_mask':mask_tgt, 'encoder_out':test_src, 'encoder_mask':mask_src})
out['input'].shape

In [ ]:
official_decoder_layer = nn.TransformerDecoderLayer(512, 8, 2048, batch_first=True, dropout=0)
official_decoder_layer.self_attn.in_proj_weight.data = decoder_layer.att_block.submodule.qkv.weight.data
official_decoder_layer.self_attn.in_proj_bias.data = decoder_layer.att_block.submodule.qkv.bias.data
official_decoder_layer.self_attn.out_proj.weight.data = decoder_layer.att_block.submodule.out_proj.weight.data
official_decoder_layer.self_attn.out_proj.bias.data = decoder_layer.att_block.submodule.out_proj.bias.data
official_decoder_layer.multihead_attn.in_proj_weight.data = decoder_layer.cross_att_block.submodule.qkv.weight.data
official_decoder_layer.multihead_attn.in_proj_bias.data = decoder_layer.cross_att_block.submodule.qkv.bias.data
official_decoder_layer.multihead_attn.out_proj.weight.data = decoder_layer.cross_att_block.submodule.out_proj.weight.data
official_decoder_layer.multihead_attn.out_proj.bias.data = decoder_layer.cross_att_block.submodule.out_proj.bias.data
official_decoder_layer.linear1.weight.data = decoder_layer.mlp_block.submodule.layer[0].weight.data
official_decoder_layer.linear1.bias.data = decoder_layer.mlp_block.submodule.layer[0].bias.data
official_decoder_layer.linear2.weight.data = decoder_layer.mlp_block.submodule.layer[2].weight.data
official_decoder_layer.linear2.bias.data = decoder_layer.mlp_block.submodule.layer[2].bias.data
official_decoder_layer.norm1.weight.data = decoder_layer.att_block.layer_norm.weight.data
official_decoder_layer.norm1.bias.data = decoder_layer.att_block.layer_norm.bias.data
official_decoder_layer.norm2.weight.data = decoder_layer.cross_att_block.layer_norm.weight.data
official_decoder_layer.norm2.bias.data = decoder_layer.cross_att_block.layer_norm.bias.data
official_decoder_layer.norm3.weight.data = decoder_layer.mlp_block.layer_norm.weight.data
official_decoder_layer.norm3.bias.data = decoder_layer.mlp_block.layer_norm.bias.data

# Mask for self-attentionq
# In nn.TransformerEncoderLayer or nn.TransformerDecoderLayer, the mask is expected to be [N, Tq, Tk]
# where N is the batch size, Tt is the query sequence length, and Ts is the key sequence length.
head_repeated_mask_tgt = mask_tgt.unsqueeze(1).repeat(1,8,1,1).reshape(-1, mask_tgt.shape[1], mask_tgt.shape[2])
official_decoder_output = official_decoder_layer(test_tgt, test_src, tgt_mask=head_repeated_mask_tgt==0, memory_key_padding_mask=mask_src==0)

assert torch.allclose(official_decoder_output, out['input'], atol=1e-4), "Your output is different from the official output"
print('Passed the test cases!')

### Using your implementation, we can build Transformer!

In [ ]:
class Encoder(nn.Module):
  def __init__(self, in_size, emb_size, mlp_size, num_head, num_layers, vocab_size):
    super().__init__()
    self.layers = nn.Sequential()
    for i in range(num_layers):
      self.layers.append(EncoderLayer(in_size,emb_size,mlp_size,num_head))
    self.pos_enc = PosEncoding(emb_size, 10000)
    self.token_emb = nn.Embedding(vocab_size, emb_size)

  def forward(self, x):
    mask = torch.ones([x.shape[0], x.shape[1]])
    mask[x==0] = 0
    temp = torch.ones_like(x)
    result = torch.arange(x.shape[-1]).to(x.device) * temp
    x = self.token_emb(x) + self.pos_enc(result)
    return self.layers({'input':x, 'mask':mask})


class Decoder(nn.Module):
  def __init__(self, in_size, emb_size, mlp_size, num_head, num_layers, vocab_size):
    super().__init__()
    self.layers = nn.Sequential()
    for i in range(num_layers):
      self.layers.append(DecoderLayer(in_size,emb_size,mlp_size,num_head))
    self.pos_enc = PosEncoding(emb_size, 10000)
    self.token_emb = nn.Embedding(vocab_size, emb_size)

  def forward(self, x, y):
    mask = torch.tril(torch.ones(x.shape[0], x.shape[1], x.shape[1]))

    temp = torch.ones_like(x)
    result = torch.arange(x.shape[-1]).to(x.device) * temp
    x = self.token_emb(x) + self.pos_enc(result)
    return self.layers({'input':x, 'decoder_mask':mask, 'encoder_out':y['input'], 'encoder_mask':y['mask']})


class TransformerTranslator(nn.Module):
  def __init__(self, in_size, emb_size, mlp_size, num_head, num_enc_layers, num_dec_layers, enc_vocab_size, dec_vocab_size):
    super().__init__()
    self.encoder = Encoder(in_size, emb_size, mlp_size, num_head, num_enc_layers, enc_vocab_size)
    self.decoder = Decoder(in_size, emb_size, mlp_size, num_head, num_dec_layers, dec_vocab_size)
    self.final_proj = nn.Linear(emb_size, dec_vocab_size)

  def forward(self, x:torch.Tensor, y:torch.Tensor):
    '''
    Arguments:
    '''
    enc_out = self.encoder(x)
    dec_out = self.decoder(y, enc_out)
    return self.final_proj(dec_out['input']).softmax(dim=-1)

class TransformerTrainer(Trainer):
  def __init__(self, model, optimizer, loss_fn, train_loader, valid_loader, device):
    super().__init__(model, optimizer, loss_fn, train_loader, valid_loader, device)
    self.num_iter = 0
    self._adjust_optim()

  def _adjust_optim(self):
    self.num_iter += 1
    self.optimizer.param_groups[0]['lr'] = 512 ** (-0.5) * min(self.num_iter**(-0.5), self.num_iter*4000**(-1.5))

  def _train_by_single_batch(self, batch):
    '''
    This method updates self.model's parameter with a given batch

    batch (tuple): (batch_of_input_text, batch_of_label)

    You have to use variables below:

    self.model (Translator/torch.nn.Module): A neural network model
    self.optimizer (torch.optim.adam.Adam): Adam optimizer that optimizes model's parameter
    self.loss_fn (function): function for calculating BCE loss for a given prediction and target
    self.device (str): 'cuda' or 'cpu'

    output: loss (float): Mean binary cross entropy value for every sample in the training batch
    The model's parameters, optimizer's steps has to be updated inside this method
    '''

    src, tgt_i, tgt_o = batch
    pred = self.model(src.to(self.device), tgt_i.to(self.device))
    pred = pack_padded_sequence(pred, pad_packed_sequence(tgt_o)[1], batch_first=True, enforce_sorted=False)
    loss = self.loss_fn(pred.data, tgt_o.data)
    loss.backward()
    self.optimizer.step()
    self.optimizer.zero_grad()
    self._adjust_optim()

    return loss.item()

  def validate(self, external_loader=None):
    '''
    This method calculates accuracy and loss for given data loader.
    It can be used for validation step, or to get test set result

    input:
      data_loader: If there is no data_loader given, use self.valid_loader as default.


    output:
      validation_loss (float): Mean Binary Cross Entropy value for every sample in validation set
      validation_accuracy (float): Mean Accuracy value for every sample in validation set

    '''

    if external_loader and isinstance(external_loader, DataLoader):
      loader = external_loader
      print('An arbitrary loader is used instead of Validation loader')
    else:
      loader = self.valid_loader

    self.model.eval()


    validation_loss = 0
    num_correct_guess = 0
    num_data = 0
    with torch.inference_mode():
      for batch in tqdm(loader, leave=False):
        src, tgt_i, tgt_o = batch
        tgt_o = tgt_o.to(self.device)
        pred = self.model(src.to(self.device), tgt_i.to(self.device))
        pred = pack_padded_sequence(pred, pad_packed_sequence(tgt_o)[1], batch_first=True, enforce_sorted=False)
        loss = self.loss_fn(pred.data, tgt_o.data)

        if isinstance(pred, PackedSequence):
          loss = self.loss_fn(pred.data, tgt_o.data)
        else:
          loss = self.loss_fn(pred, tgt_o)

        validation_loss += loss.item() * len(pred.data)
        if isinstance(pred, PackedSequence):
          num_correct_guess += (pred.data.argmax(dim=-1) == tgt_o.data).sum().item()
        else:
          num_correct_guess += (pred.argmax(dim=-1) == tgt_o.data).sum().item()
        num_data += len(pred.data)
    return validation_loss / num_data, num_correct_guess / num_data

def pad_collate(raw_batch):
  srcs = [x[0] for x in raw_batch]
  tgts_i = [x[1][:-1] for x in raw_batch]
  tgts_o = [x[1][1:] for x in raw_batch]

  srcs = pad_sequence(srcs, batch_first=True)
  tgts_i = pad_sequence(tgts_i, batch_first=True)
  tgts_o = pack_sequence(tgts_o, enforce_sorted=False)
  return srcs, tgts_i, tgts_o

single_loader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)
train_loader = DataLoader(trainset, batch_size=4, collate_fn=pad_collate, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(validset, batch_size=128, collate_fn=pad_collate, shuffle=False, num_workers=0, pin_memory=True)
test_loader = DataLoader(testset, batch_size=128, collate_fn=pad_collate, shuffle=False, num_workers=0, pin_memory=True)

batch = next(iter(train_loader))
tfm_model = TransformerTranslator(512,512,2048,8,6,12,32000,32000)
out = tfm_model(batch[0], batch[1])


#### Transformer Test

In [ ]:
# download pre-trained weight
!gdown 1nxbQPD4a2SVicLBfzrj2pRU4hAapeMmL

In [ ]:
tfm_model = TransformerTranslator(512,512,2048,8,6,12,32000,32000)
tfm_model.load_state_dict(torch.load('kor_eng_translator_tfm_model_best.pt')['model'])

In [ ]:
def translate_tfm(model, source_sentence, src_tokenizer, tgt_tokenizer):
  '''

  Arguments:
    model (TranslatorAtt): Translator model with attention
    source_sentence (str): Sentence to translate

  Returns:
    input_tokens (list): Source sentence in a list of token in token_id
    predicted_tokens (list): Translated sentence in a list of token in token_id
    decoded_string (str): Translated sentence in string

  '''

  input_tokens = src_tokenizer.encode(source_sentence)
  input_tensor = torch.LongTensor(input_tokens).unsqueeze(0)
  enc_out = model.encoder(input_tensor)

  # Setup for 0th step
  current_decoder_token = torch.LongTensor([[2]]) # start of sentence token

  for i in range(50): # You can chage it to while True:
    decoder_out = model.decoder(current_decoder_token, enc_out)['input']
    logit = model.final_proj(decoder_out[0, -1])
    selected_token = torch.argmax(logit, dim=-1)
    current_decoder_token = torch.tensor(current_decoder_token[0].tolist() + [selected_token], dtype=torch.long).unsqueeze(0)
    if selected_token == 3: ## end of sentence token
      break
  predicted_tokens = current_decoder_token.squeeze().tolist()[1:-1]
  return input_tokens, predicted_tokens, tgt_tokenizer.decode(predicted_tokens)

tfm_model.cpu()
tfm_model.eval()
input_tokens, pred_tokens, translated_string  = translate_tfm(tfm_model, '문장이 잘 번역되는지를 체크함으로써 모델 설계와 학습이 잘 이뤄졌는지를 확인할 수 있습니다.', src_tokenizer, tgt_tokenizer)
print(f"Input: {src_tokenizer.decode(input_tokens)}")
print(f"Output: {tgt_tokenizer.decode(pred_tokens)}")


# Check Before Submission
- Copy and paste your code to the downloaded ``NLP_Assignment_4.py``
  - https://raw.githubusercontent.com/jdasam/aat3020/2025/NLP_Assignment_4.py

In [ ]:
# Run this code after copy and paste your code to NLP_Assignment_4.py
!python NLP_Assignment_4.py